### Code Written by- Asif Rana, Student ID-102866893

In [1]:
# Load the Python libraries
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
import time
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn.metrics as sm
from sklearn.metrics import mean_absolute_error
import time

# Load Surprise libraries
from surprise import SVD
from surprise import Reader
from surprise import Dataset
from surprise import accuracy

## Using Content-based Recommendation 

In [2]:
# Importing the csv data files 
movies_metadata = pd.read_csv('metadata_integrated.csv',error_bad_lines=False, warn_bad_lines=True)
train_df = pd.read_csv('train_df.csv',error_bad_lines=False, warn_bad_lines=True)
movie_title = movies_metadata[['movieId','title']]

In [3]:
movie_title.head()

,movieId,title
0,1,Toy Story
1,2,Jumanji
2,3,Grumpier Old Men
3,4,Waiting to Exhale
4,5,Father of the Bride Part II


In [4]:
global_mean_rating = np.average(train_df.rating.values)

In [5]:
global_mean_rating

3.5399152683169963

# Change the filename below to your required csv file

In [6]:
#Load required csv file
test = pd.read_csv('unrated_movies.csv',error_bad_lines=False, warn_bad_lines=True)
test_df = test 
test_df1 = test 

## Conducting the preprocessing required

In [7]:
content_recom_df = movies_metadata[['movieId', 'overview','genres','title', 'production_companies', 'collectionName']]
content_recom_df['genres'] = content_recom_df['genres'].str.replace(',',' ').str.lower()
content_recom_df['concat_title'] = content_recom_df['title'].str.lower().str.replace(' ','') + content_recom_df['movieId'].astype(str)
content_recom_df['overview'] = content_recom_df['overview'].str.lower()
content_recom_df['collectionName'] = content_recom_df['collectionName'].str.replace(' ','').str.lower()
content_recom_df['production_companies'] = content_recom_df['production_companies'].str.replace(' ','').str.lower().str.replace(',',' ')

content_recom_df['corpus'] =  content_recom_df['genres'] + ' ' + content_recom_df['overview']  
content_recom_df['corpus'] = content_recom_df['corpus'].str.replace('[^\w\s]','') 
content_recom_df['corpus'] = content_recom_df['corpus'].str.replace('unknown','')

<ipython-input-7-4a909ba708df>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content_recom_df['genres'] = content_recom_df['genres'].str.replace(',',' ').str.lower()
<ipython-input-7-4a909ba708df>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content_recom_df['concat_title'] = content_recom_df['title'].str.lower().str.replace(' ','') + content_recom_df['movieId'].astype(str)
<ipython-input-7-4a909ba708df>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

In [8]:
#Define a TF-IDF Vectorizer Object. Removing all english stopwords
tfidf = TfidfVectorizer(stop_words='english')

#Constructing the required TF-IDF matrix by applying the fit_transform method on the overview feature
tfidf_matrix = tfidf.fit_transform(content_recom_df['corpus'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

# Import linear_kernel to compute the dot product
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim_tfidf = linear_kernel(tfidf_matrix, tfidf_matrix)

In [9]:
# Reset index of your df and construct reverse mapping again
content_recom_df = content_recom_df.reset_index()
indices = pd.Series(content_recom_df.index, index=content_recom_df['movieId'])

In [10]:
train_df = train_df[['userId','movieId','rating']]
test_df1 = test_df1[['userId','movieId',]]

In [11]:
test_df1.head()

,userId,movieId
0,1,1
1,1,2
2,1,3
3,1,4
4,1,5


In [12]:
#5967523 entries
start = time.time()
users = test_df1.userId.unique()
k = 36 # Setting the value for the most similar movies number
start = time.time()
for i in range(len(users)):
    start1 = time.time()
    rated_movies = train_df.loc[(train_df['userId'] == users[i])].movieId.values
    unrated_movies = test_df1.loc[(test_df1['userId'] == users[i])].movieId.values
    for movie in rated_movies:
        idx = indices[movie]
        sim_scores = list(enumerate(cosine_sim_tfidf[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:k]
        movie_indices = [i[0] for i in sim_scores]
        movieIds = movies_metadata['movieId'].iloc[movie_indices].values    
        unrated_movies = list(set(movieIds).intersection(unrated_movies))
        if len(unrated_movies) > 0:
            for movie in unrated_movies:
                idx = indices[movie]
                sim_scores = list(enumerate(cosine_sim_tfidf[idx]))
                sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
                sim_scores = sim_scores[1:k]
                movieIds = movies_metadata['movieId'].iloc[movie_indices].values  
                movie_indices = [i[0] for i in sim_scores]
                
                movieIds = movies_metadata['movieId'].iloc[movie_indices].values  
                unrated_movies = list(set(movieIds).intersection(rated_movies))
                if len(unrated_movies)> 0:
                    weight_movies = [i[1] for i in sim_scores]
                    temp = pd.DataFrame()
                    temp2 = pd.DataFrame()
                    merged_rating_df = pd.DataFrame()
                    temp['movieId'] = movies_metadata['movieId'].iloc[movie_indices]
                    temp['weights'] = weight_movies
                    temp2['movieId'] = unrated_movies 
                    merged_rating_df = pd.merge(temp, temp2, how='inner', left_on=['movieId'], right_on = ['movieId'])
                    merged_rating_df = pd.merge(train_df.loc[(train_df['userId'] == users[i])], merged_rating_df , how='inner', left_on=['movieId'], right_on = ['movieId'])
                    merged_rating_df["weights*rating"] = merged_rating_df["weights"] * merged_rating_df["rating"]
                    weights = merged_rating_df['weights'].values
                    weights_rating = merged_rating_df['weights*rating'].values
                    prediction = sum(weights_rating)/(sum(weights))
                    test_df1.loc[(test_df1['userId'] == users[i]) & (test_df1['movieId'] == movie),'prediction'] = prediction     
    end1 = time.time()
    print('User : ', users[i])
    print(end1 - start1)  
end = time.time()
print(end - start)       

User :  1
2.163212776184082
User :  7
2.989008665084839
User :  31
2.66288423538208
User :  32
2.1542372703552246
User :  36
3.4433703422546387
User :  39
2.8683362007141113
User :  73
11.324675559997559
User :  88
5.1877641677856445
User :  96
2.3128130435943604
User :  110
3.926499843597412
User :  111
5.450430631637573
User :  150
5.528814077377319
User :  161
4.111979007720947
User :  165
6.437423944473267
User :  186
2.1492133140563965
User :  242
5.548165321350098
User :  254
3.3889801502227783
User :  288
3.6900923252105713
User :  310
2.024229049682617
User :  311
8.461377143859863
User :  325
2.0156075954437256
User :  338
3.9015753269195557
User :  341
2.288393974304199
User :  386
4.852024078369141
User :  394
4.306482791900635
User :  452
9.649780511856079
User :  457
7.581768989562988
User :  461
7.212753772735596
User :  468
9.08970022201538
User :  485
1.667539119720459
User :  487
4.67909574508667
User :  496
4.743459701538086
User :  511
2.1522490978240967
User :  516


User :  381
4.104032278060913
User :  412
3.9364681243896484
User :  480
5.104350328445435
User :  557
3.1859493255615234
User :  595
3.5874481201171875
User :  611
3.482689142227173
User :  656
3.63132643699646
User :  75
3.7539634704589844
User :  157
5.543738603591919
User :  167
3.3600211143493652
User :  199
5.337725639343262
User :  355
4.7677295207977295
User :  395
3.2762773036956787
User :  482
3.032895565032959
User :  544
5.009669542312622
User :  599
4.3759753704071045
User :  626
3.519583225250244
User :  68
4.542858362197876
User :  282
3.931469202041626
User :  76
2.41754150390625
User :  83
3.8402867317199707
User :  125
4.273607969284058
User :  134
4.60170316696167
User :  178
3.4712865352630615
User :  196
4.085083961486816
User :  222
4.291526556015015
User :  315
2.514281749725342
User :  352
3.2403368949890137
User :  398
1.5049738883972168
User :  424
2.2954044342041016
User :  519
2.707808494567871
User :  553
3.4636902809143066
User :  615
5.153255939483643
Use

User :  218
3.848714828491211
User :  230
2.8390040397644043
User :  612
2.135283946990967
User :  60
4.123978614807129
User :  348
2.512286901473999
User :  393
4.126920938491821
User :  644
1.5743911266326904
User :  12
3.765970468521118
User :  189
3.8038666248321533
User :  215
2.711712598800659
User :  368
2.9750795364379883
User :  645
3.244858980178833
User :  515
4.366324424743652
User :  107
1.7843339443206787
User :  112
2.159428596496582
User :  181
2.4025726318359375
User :  208
2.0066325664520264
User :  274
4.7451512813568115
User :  278
2.9641149044036865
User :  334
2.33076810836792
User :  445
2.2090537548065186
User :  506
2.002681016921997
User :  524
3.3041727542877197
User :  583
1.913468360900879
User :  609
3.3898932933807373
User :  668
1.953162670135498
User :  210
3.499682664871216
User :  158
3.222382068634033
User :  127
4.131544589996338
User :  160
3.9833555221557617
User :  131
3.012568235397339
User :  140
3.979358673095703
User :  166
3.240337610244751


In [13]:
recom_content_pred = test_df1

In [14]:
recom_content_pred = recom_content_pred.fillna(global_mean_rating)

In [15]:
recom_content_pred.prediction.unique()

array([3.53991527, 2.5       , 2.5       , ..., 1.90398833, 4.58789939,
       3.48196351])

In [16]:
recom_content_pred = pd.merge(recom_content_pred, movie_title, how='left', left_on=['movieId'], right_on = ['movieId'])

In [17]:
sorted_pred = recom_content_pred.sort_values(by =['userId', 'prediction'], ascending= False)

In [18]:
sorted_pred.head()

,userId,movieId,prediction,title
2692778,671,490,5.0,Malice
2694094,671,2269,5.0,Indecent Proposal
2694492,671,2750,5.0,Radio Days
2694708,671,3021,5.0,The Funhouse
2694894,671,3263,5.0,White Men Can't Jump


## Printing the recommendations for Content-Based

In [19]:
for i in range(len(users)):
    top10movies = sorted_pred.loc[(sorted_pred['userId'] == users[i])]['title'].values
    top10movies = top10movies[:10]
    print('Recommendation for user: ', users[i])
    print('Movies: ',top10movies)

Recommendation for user:  1
Movies:  ['Toy Story' 'Jumanji' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Heat' 'Sabrina' 'Tom and Huck'
 'Sudden Death' 'GoldenEye']
Recommendation for user:  7
Movies:  ['Jumanji' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Heat' 'Sabrina' 'Tom and Huck'
 'Sudden Death' 'The American President' 'Dracula: Dead and Loving It']
Recommendation for user:  31
Movies:  ['My Super Ex-Girlfriend' 'Jason and the Argonauts'
 'The Bridges of Madison County' 'Jack & Sarah' 'Pollyanna'
 'Blood, Guts, Bullets and Octane' 'To Sir, with Love' 'Elephant'
 'Only the Strong' 'Born Rich']
Recommendation for user:  32
Movies:  ['The Paleface' 'Pathfinder' 'Ong-Bak: The Thai Warrior' 'Toy Story'
 'Grumpier Old Men' 'Waiting to Exhale' 'Father of the Bride Part II'
 'Heat' 'Sabrina' 'Tom and Huck']
Recommendation for user:  36
Movies:  ['Star Trek V: The Final Frontier' 'Roadgames' 'İtirazım Var'
 'Perfect Stranger' 'Manhunt

Recommendation for user:  175
Movies:  ['Malice' 'Rebel Without a Cause' 'Condorman' 'Indecent Proposal'
 'Radio Days' 'The Funhouse' "White Men Can't Jump" 'Bound for Glory'
 "The Ballad of Ramblin' Jack" "Max Keeble's Big Move"]
Recommendation for user:  185
Movies:  ['Goodbye, Columbus' 'Rebel Without a Cause' 'Indecent Proposal'
 'The Thin Red Line' 'Stepmom' 'Radio Days' 'The Funhouse'
 "White Men Can't Jump" 'Bound for Glory' "What's Up, Tiger Lily?"]
Recommendation for user:  188
Movies:  ['Guardians of the Galaxy' 'D3: The Mighty Ducks' 'eXistenZ'
 'Any Given Sunday' 'Dungeons & Dragons' 'Manhunter' 'Panic Room'
 'Rollerball' 'Spy Kids 3-D: Game Over' 'Avalon']
Recommendation for user:  213
Movies:  ['Rebel Without a Cause' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Heat' 'Tom and Huck' 'Sudden Death'
 'GoldenEye' 'The American President' 'Dracula: Dead and Loving It']
Recommendation for user:  214
Movies:  ['Eye of the Needle'
 'The Assassination of

Recommendation for user:  584
Movies:  ['Goodbye, Columbus' 'Malice' 'Indecent Proposal' 'The Thin Red Line'
 'Stepmom' 'Radio Days' 'The Funhouse' 'Toy Story 2' 'Hollywood Ending'
 "What's Up, Tiger Lily?"]
Recommendation for user:  608
Movies:  ['Hollywood Ending' 'Mask' 'The Messenger' 'Rebel Without a Cause'
 'Stepmom' 'The Funhouse' 'Africa Screams' 'Goodbye, Columbus'
 'Tai-Chi Master' 'For Your Consideration']
Recommendation for user:  647
Movies:  ['The Thin Red Line' "Max Keeble's Big Move" 'Maid in Manhattan'
 'Goodbye, Columbus' 'Room' 'Bound for Glory' 'Tai-Chi Master' 'Malice'
 'Rebel Without a Cause' 'Condorman']
Recommendation for user:  21
Movies:  ['The Living Daylights' 'Game of Death' 'The Man from U.N.C.L.E.'
 'Toy Story' 'Jumanji' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Heat' 'Sabrina']
Recommendation for user:  34
Movies:  ['Benny & Joon' 'Anaconda' 'The Hurt Locker' 'Night Moves' 'Biloxi Blues'
 'The Trial' 'Falling Down' 'Mulhollan

Movies:  ['The Thin Red Line' 'Goodbye, Columbus'
 'Stanley Kubrick: A Life in Pictures' "White Men Can't Jump"
 'Maid in Manhattan' 'Rudolph, the Red-Nosed Reindeer' 'Match Point'
 "Max Keeble's Big Move" 'Bound for Glory' 'The Messenger']
Recommendation for user:  441
Movies:  ['The Longest Week' 'Poetic Justice' 'Shadowlands' 'Diva' 'Woo'
 "Captain Corelli's Mandolin" 'Birthday Girl' 'Houseboat'
 'Ali: Fear Eats the Soul' 'Hells Angels on Wheels']
Recommendation for user:  460
Movies:  ['Dirty Work' 'Rebel Without a Cause' 'The Funhouse'
 "The Ballad of Ramblin' Jack" "What's Up, Tiger Lily?"
 'For Your Consideration' 'Let It Ride' 'Africa Screams' 'Match Point'
 'Bound for Glory']
Recommendation for user:  481
Movies:  ['Stanley Kubrick: A Life in Pictures' 'A Midnight Clear'
 "What's Up, Tiger Lily?" 'Mask' "The Ballad of Ramblin' Jack"
 'Rudolph, the Red-Nosed Reindeer' "Max Keeble's Big Move"
 'Maid in Manhattan' 'Bound for Glory' 'Smiles of a Summer Night']
Recommendation for u

Recommendation for user:  247
Movies:  ['Goodbye, Columbus' 'The Thin Red Line' 'Room' 'Dirty Work' 'Malice'
 'Rebel Without a Cause' 'Radio Days' 'The Funhouse'
 "The Ballad of Ramblin' Jack" "Max Keeble's Big Move"]
Recommendation for user:  285
Movies:  ['Goodbye, Columbus' "The Ballad of Ramblin' Jack" 'Radio Days' 'Malice'
 'Rebel Without a Cause' 'The Funhouse' 'Let It Ride'
 "Max Keeble's Big Move" 'Rudolph, the Red-Nosed Reindeer'
 'Hollywood Ending']
Recommendation for user:  290
Movies:  ['For Your Eyes Only' 'The Living Daylights' 'Brainstorm'
 'Lara Croft Tomb Raider: The Cradle of Life' 'Bunraku'
 'The Way of the Dragon' 'Dr. No' 'A View to a Kill' 'Licence to Kill'
 "On Her Majesty's Secret Service"]
Recommendation for user:  297
Movies:  ['Mallrats' 'Toy Story' 'Jumanji' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Heat' 'Sabrina' 'Tom and Huck'
 'Sudden Death']
Recommendation for user:  300
Movies:  ['Days of Thunder' 'Outbreak' 'The Glimmer Ma

...

Recommendation for user:  11
Movies:  ['Like Water for Chocolate' 'Daylight' 'The Sting' 'Still Breathing'
 'The Dangerous Lives of Altar Boys' 'Throne of Blood' 'Four Lions'
 'Hotel Chevalier' 'Nightcrawler' 'Odds Against Tomorrow']
Recommendation for user:  16
Movies:  ['Like Water for Chocolate' 'Daylight' 'Still Breathing'
 'The Dangerous Lives of Altar Boys' 'Odds Against Tomorrow'
 'Throne of Blood' 'Four Lions' 'Hotel Chevalier' 'Nightcrawler'
 'Toy Story']
Recommendation for user:  40
Movies:  ['The Thief of Bagdad' 'Sinbad and the Eye of the Tiger'
 'The Swan Princess' 'Princess Caraboo' 'Return of the Jedi'
 'History of the World: Part I'
 'Star Wars: Episode I - The Phantom Menace' 'The Mummy'
 'The 7th Voyage of Sinbad' 'Shanghai Noon']
Recommendation for user:  97
Movies:  ['Smiles of a Summer Night' 'Jumanji' 'Grumpier Old Men'
 'Waiting to Exhale' 'Father of the Bride Part II' 'Heat' 'Sabrina'
 'Tom and Huck' 'Sudden Death' 'GoldenEye']
Recommendation for user:  138
Movi

Recommendation for user:  237
Movies:  ['Jumanji' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Heat' 'Sabrina' 'Tom and Huck'
 'Sudden Death' 'GoldenEye' 'The American President']
Recommendation for user:  244
Movies:  ['Mrs. Miniver' 'Tea with Mussolini' 'Saints and Soldiers'
 'The Last of the Mohicans' 'The Lords of Flatbush'
 'Village of the Damned' 'Separate Lies' 'Toy Story' 'Jumanji'
 'Grumpier Old Men']
Recommendation for user:  251
Movies:  ['Mirror' 'Broken' 'King Ralph' "The Blood on Satan's Claw" 'Henry V'
 'Village of the Damned' 'Separate Lies' 'The Lords of Flatbush'
 'A Close Shave' 'Stalingrad']
Recommendation for user:  277
Movies:  ["Soldier's Girl" 'A Close Shave' 'Stalingrad' 'Henry V'
 'Breaking the Waves' 'The Last of the Mohicans' 'Village of the Damned'
 'The Lords of Flatbush' 'Scandal' 'Straw Dogs']
Recommendation for user:  293
Movies:  ['Love! Valour! Compassion!' 'Malice' 'Rebel Without a Cause' 'Condorman'
 'Indecent Proposal' 'Ra

Recommendation for user:  230
Movies:  ['Harold and Maude' 'D2: The Mighty Ducks' 'The Mighty Ducks'
 'The Big Store' 'Hello Ladies: The Movie' 'To Sleep with Anger'
 'Detroit Rock City' 'Ghost Dad' 'Brief Encounter' 'Empire Records']
Recommendation for user:  612
Movies:  ['Waiting...' 'Toy Story' 'Jumanji' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Heat' 'Sabrina' 'Tom and Huck'
 'Sudden Death']
Recommendation for user:  60
Movies:  ['Black Rain' 'Leaving Las Vegas' 'Funny Bones' 'Think Like a Man Too'
 'Bugsy' 'Showgirls' 'Go' 'Last Vegas' 'Little Odessa' 'Donnie Brasco']
Recommendation for user:  348
Movies:  ['Caught' 'Carrie' 'The Butterfly Effect 3: Revelations'
 'The Serpent and the Rainbow' 'Toy Story' 'Jumanji' 'Grumpier Old Men'
 'Waiting to Exhale' 'Father of the Bride Part II' 'Heat']
Recommendation for user:  393
Movies:  ['Funny Bones' 'Toy Story' 'Jumanji' 'Grumpier Old Men'
 'Waiting to Exhale' 'Father of the Bride Part II' 'Heat' 'Sabrina'


Recommendation for user:  498
Movies:  ['Toy Story' 'Jumanji' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Heat' 'Sabrina' 'Tom and Huck'
 'Sudden Death' 'GoldenEye']
Recommendation for user:  37
Movies:  ['The Chronicles of Narnia: Prince Caspian' 'Flipper' 'The Wizard of Oz'
 'Meet the Deedles' 'Quest for Camelot' 'A Perfect Murder'
 'The Addams Family' 'Shadow of a Doubt' 'The Bear' 'Benji the Hunted']
Recommendation for user:  136
Movies:  ['Malice' 'Rebel Without a Cause' 'Condorman' 'Indecent Proposal'
 'Radio Days' 'The Funhouse' "White Men Can't Jump" 'Bound for Glory'
 'Hollywood Ending' "What's Up, Tiger Lily?"]
Recommendation for user:  359
Movies:  ['La Dolce Vita' 'Poetic Justice' 'Shadowlands' 'Diva' 'Woo'
 'The Effect of Gamma Rays on Man-in-the-Moon Marigolds' 'Nowhere to Hide'
 "Captain Corelli's Mandolin" 'Shallow Hal' 'Birthday Girl']
Recommendation for user:  101
Movies:  ["Charlie's Angels: Full Throttle" 'Toy Story' 'Jumanji'
 'Grumpier O

Movies:  ['Malice' 'Rebel Without a Cause' 'Condorman' 'Indecent Proposal'
 'Radio Days' 'The Funhouse' 'Toy Story 2' "White Men Can't Jump"
 'Bound for Glory' 'Hollywood Ending']
Recommendation for user:  490
Movies:  ['The Van' 'The Man Who Knew Too Little' 'Inspector Gadget'
 'Double Impact' 'Youngblood' 'Kickboxer' 'National Lampoon’s Van Wilder'
 'Baadasssss!' 'The Love Guru' 'Jumanji']
Recommendation for user:  29
Movies:  ["Carlito's Way" 'Tequila Sunrise' 'Fatal Beauty' 'Traffic' 'Brubaker'
 'The Principal' 'Journey to the Center of the Earth' 'Walking Tall'
 'The Prime of Miss Jean Brodie' "Bush's Brain"]
Recommendation for user:  179
Movies:  ['Malice' 'Rebel Without a Cause' 'Condorman' 'Indecent Proposal'
 'Radio Days' 'The Funhouse' "White Men Can't Jump" 'Bound for Glory'
 'Hollywood Ending' 'Maid in Manhattan']
Recommendation for user:  374
Movies:  ['Toy Story' 'Jumanji' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Heat' 'Sabrina' 'Tom and Huck

## Collaborative Filters

### Creating datasets for collaborative filters

In [20]:
#Remove all information except User ID, Movie ID and Title
test_data = test[['userId','movieId','rating']]

In [21]:
# Read the data into a Surprise dataset
reader = Reader(rating_scale = (1, 5))
data_test = Dataset.load_from_df(test_data, reader)

# Build full trainset
data_test = data_test.build_full_trainset()

# Create the trainset and testset
data_testset = data_test.build_testset()

# Model-Based

In [22]:
#loading model pickle file
filename1 = 'model-based_CF.pickle'
model_cf = pickle.load(open(filename1, 'rb'))

In [23]:
# Predicting rating for the dataset
pred_rating = model_cf.test(data_testset)

In [24]:
# Saving predictions 
predictions = []
for i in range(len(pred_rating)):
    temp = pred_rating[i].est
    temp1 = round(temp,2)
    predictions.append(temp1)

In [25]:
# Converting to dataframe
predictions_df = pd.DataFrame(predictions, columns=['predictions'])

In [26]:
# Loading predictions onto dataset
model_cf_df = test_data.join(predictions_df)

In [27]:
#Dropping previous 'rating' column
model_cf_df  = model_cf_df.drop(columns=['rating'])

In [28]:
model_cf_df

,userId,movieId,predictions
0,1,1,2.72
1,1,2,2.41
2,1,3,2.15
3,1,4,1.00
4,1,5,1.98
...,...,...,...
5967518,227,161944,4.44
5967519,227,162542,4.05
5967520,227,162672,3.82
5967521,227,163056,4.05


### Printing the recommendations for Model-Based Collaborative Filtering

In [29]:
model_cf_df = pd.merge(model_cf_df, movie_title, how='left', left_on=['movieId'], right_on = ['movieId'])

In [30]:
sorted_pred = model_cf_df.sort_values(by =['userId', 'predictions'], ascending= False)

In [31]:
users = model_cf_df.userId.unique()
for i in range(len(users)):
    top10movies = sorted_pred.loc[(sorted_pred['userId'] == users[i])]['title'].values
    top10movies = top10movies[:10]
    print('Recommendation for user: ', users[i])
    print('MovieIds: ',top10movies)

Recommendation for user:  1
MovieIds:  ['Breathless' 'El Topo' 'Aguirre: The Wrath of God' 'The Endless Summer'
 'Titus' 'Blue in the Face' 'A Walk on the Moon' 'Cyclo' 'Out of the Past'
 'Fitzcarraldo']
Recommendation for user:  7
MovieIds:  ['Land of Silence and Darkness' 'The Play House' 'The Goat' 'Cops'
 'The Last Detail' 'Holiday' 'Steamboat Willie' 'The Endless Summer'
 'Sherlock, Jr.' 'The Godfather']
Recommendation for user:  31
MovieIds:  ['Naked' 'Drunken Master' 'Crimes and Misdemeanors' 'Titus' 'Breathless'
 'The Magnificent Ambersons' 'Nowhere in Africa'
 'Aguirre: The Wrath of God' 'Dear Frankie' 'El Topo']
Recommendation for user:  32
MovieIds:  ['Killing Zoe' 'Dead Man' 'The Ruling Class' 'Local Hero' 'Sliding Doors'
 'Wild Man Blues' 'And the Band Played On' 'The Gods Must Be Crazy II'
 'Man Facing Southeast' 'Crimes and Misdemeanors']
Recommendation for user:  36
MovieIds:  ['Nights of Cabiria' 'Titus' "Hachi: A Dog's Tale" 'Dangerous Beauty'
 'Aparajito' 'Faster, Pu

Recommendation for user:  185
MovieIds:  ['Faster, Pussycat! Kill! Kill!'
 'Thirty Two Short Films About Glenn Gould' 'Jean de Florette'
 'The Return of Martin Guerre' 'Paris is Burning' 'A Walk on the Moon'
 'Man Facing Southeast' 'Titus' "No Man's Land" 'Frida']
Recommendation for user:  188
MovieIds:  ['Les Miserables' 'Faster, Pussycat! Kill! Kill!' 'Cronos'
 'Maya Lin: A Strong Clear Vision' 'Ran' 'Fanny & Alexander'
 'And the Band Played On' 'Nights of Cabiria' 'Dangerous Beauty'
 'Perfect Blue']
Recommendation for user:  213
MovieIds:  ['Love Affair' 'Vanya on 42nd Street' 'Faster, Pussycat! Kill! Kill!'
 'The Great Dictator' 'The Pillow Book' 'Wild Man Blues'
 "Cookie's Fortune" 'Mirror' '...And Justice for All' 'We Were Soldiers']
Recommendation for user:  214
MovieIds:  ['La Haine' 'Once Were Warriors' "Muriel's Wedding" 'Red Rock West'
 'Underground' 'Maya Lin: A Strong Clear Vision' 'I Shot Andy Warhol'
 'Matilda' 'The Philadelphia Story' 'It Happened One Night']
Recommenda

Recommendation for user:  507
MovieIds:  ['Love Is a Many-Splendored Thing' "Brewster's Millions"
 'Nowhere in Africa' 'Dogville' 'Gladiator 1992' 'Batman'
 'Zero Dark Thirty' 'Nights of Cabiria' 'The Intouchables' 'Hear My Song']
Recommendation for user:  509
MovieIds:  ['The Old Man and the Sea' 'Lawrence of Arabia' 'Steamboat Willie'
 'The Last Detail' 'The Last Laugh' 'Little Dieter Needs to Fly'
 'Vernon, Florida' 'Man with a Movie Camera' 'Beauty and the Beast'
 'The Navigator']
Recommendation for user:  534
MovieIds:  ['Fluke' 'Priest' 'Red Rock West' 'Faster, Pussycat! Kill! Kill!'
 'Another Stakeout' 'Killing Zoe' 'Naked' 'Hellraiser: Bloodline'
 'I Shot Andy Warhol' 'Up in Smoke']
Recommendation for user:  537
MovieIds:  ['Les Miserables' 'The White Balloon' 'La Haine' 'Braveheart'
 'Swimming with Sharks' 'Bitter Moon' 'Wyatt Earp'
 'Faster, Pussycat! Kill! Kill!' 'Cronos' 'The World of Apu']
Recommendation for user:  584
MovieIds:  ['Les Miserables' 'Swimming with Sharks' 'D

Recommendation for user:  330
MovieIds:  ['A Little Princess' 'Faster, Pussycat! Kill! Kill!'
 'The Man without a Face' 'Naked' 'Surviving the Game' 'Cronos'
 'A Family Thing' 'The Quest' 'Looking for Richard' 'The Ruling Class']
Recommendation for user:  372
MovieIds:  ['Les Miserables' 'Death and the Maiden' 'Vanya on 42nd Street'
 'Faster, Pussycat! Kill! Kill!' 'Geronimo: An American Legend'
 'Killing Zoe' 'King of the Hill'
 'Thirty Two Short Films About Glenn Gould' 'Jean de Florette'
 'The Return of Martin Guerre']
Recommendation for user:  373
MovieIds:  ['The Wild Bunch' 'Maya Lin: A Strong Clear Vision' 'The Band Wagon'
 'The Old Man and the Sea' 'Lawrence of Arabia' 'Stalker'
 'Dangerous Beauty' 'The Killing' 'Perfect Blue' 'The General']
Recommendation for user:  391
MovieIds:  ['Persuasion' 'Richard III' 'The Confessional' 'From Dusk Till Dawn'
 'The Crossing Guard' "Things to Do in Denver When You're Dead"
 "Antonia's Line" 'Chungking Express' 'Flirting with Disaster'
 'T

Recommendation for user:  236
MovieIds:  ["Things to Do in Denver When You're Dead" "Antonia's Line"
 'Living in Oblivion' 'Safe' 'Three Colors: White' 'To Live'
 'Faster, Pussycat! Kill! Kill!' 'The Endless Summer 2'
 'Surviving the Game' 'The Quest']
Recommendation for user:  281
MovieIds:  ['Three Colors: White' 'The Return of Martin Guerre' 'Paris is Burning'
 'Stalker' 'Out of the Past' 'Yojimbo' 'Mister Roberts' "No Man's Land"
 'Missing' 'The Electric Horseman']
Recommendation for user:  419
MovieIds:  ['Faster, Pussycat! Kill! Kill!' 'Across the Universe'
 'Dear Zachary: A Letter to a Son About His Father'
 'Land of Silence and Darkness' 'The Play House' 'The Goat' 'Cops'
 'Maya Lin: A Strong Clear Vision' 'The Searchers'
 'Vampire Hunter D: Bloodlust']
Recommendation for user:  532
MovieIds:  ['Faster, Pussycat! Kill! Kill!' 'Naked' 'The Great Dictator' 'Titus'
 'The Searchers' '...And Justice for All' 'Breathless' 'We Were Soldiers'
 'Frida' 'The Electric Horseman']
Recommend

...

Recommendation for user:  28
MovieIds:  ['The Confessional' 'Friday' 'The White Balloon'
 "Things to Do in Denver When You're Dead" 'Anne Frank Remembered'
 'Love & Human Remains' 'Safe' 'Burnt by the Sun' 'Picture Bride'
 'Red Firecracker, Green Firecracker']
Recommendation for user:  42
MovieIds:  ['Out of the Past' 'Airport' 'The Outlaw Josey Wales' 'The Endless Summer'
 'The Game Plan' 'Super' 'The Return of Martin Guerre' 'Beautiful Thing'
 'Auntie Mame' 'Fandango']
Recommendation for user:  64
MovieIds:  ['Naked' 'It Happened One Night' 'Dangerous Beauty' 'The Commitments'
 'Women on the Verge of a Nervous Breakdown' 'Titus' 'The Kid'
 'The Endless Summer' 'Breathless' 'The Magnificent Ambersons']
Recommendation for user:  84
MovieIds:  ['Dead Man' 'Titus' 'Breathless' 'Nowhere in Africa' 'Dear Frankie'
 'El Topo' "Hachi: A Dog's Tale" 'Love Is a Many-Splendored Thing'
 'The Dish' 'Lantana']
Recommendation for user:  89
MovieIds:  ['Persuasion' 'The Usual Suspects' 'Lamerica' 'Th

Recommendation for user:  563
MovieIds:  ['Titus' 'The Kid' 'The Endless Summer' 'Breathless'
 'Beauty and the Beast' 'Land of Silence and Darkness' 'Mary and Max'
 'The Goat' 'The Play House' 'Cops']
Recommendation for user:  565
MovieIds:  ['Faster, Pussycat! Kill! Kill!' 'The Return of Martin Guerre'
 'Paris is Burning' 'Nights of Cabiria' 'Man Facing Southeast'
 'The Story of Us' '...And Justice for All' 'Shag' "No Man's Land"
 'Across the Universe']
Recommendation for user:  571
MovieIds:  ['The White Balloon' "Things to Do in Denver When You're Dead"
 "Antonia's Line" 'Living in Oblivion' 'Faster, Pussycat! Kill! Kill!'
 'Surviving the Game' 'Cronos' 'Jean de Florette' 'Enchanted April'
 'Paris is Burning']
Recommendation for user:  657
MovieIds:  ['Women on the Verge of a Nervous Breakdown' 'The Endless Summer'
 'Out of the Past' 'Holiday' 'Breathless' 'Steamboat Willie' 'The Kid'
 'Beautiful Thing' 'The Commitments' 'Mary and Max']
Recommendation for user:  132
MovieIds:  ['Per

Recommendation for user:  506
MovieIds:  ['Titus' 'Breathless' 'El Topo' 'Benji' "Hachi: A Dog's Tale" 'Frida'
 'Aguirre: The Wrath of God' 'Drunken Master' 'Naked' 'Philomena']
Recommendation for user:  524
MovieIds:  ['Faster, Pussycat! Kill! Kill!' 'Naked' 'Enchanted April'
 'Nights of Cabiria' 'Drunken Master' 'Man Facing Southeast'
 'The Story of Us' 'Titus' 'The Ninth Gate' 'The Searchers']
Recommendation for user:  583
MovieIds:  ['Out of the Past' 'Beautiful Thing' 'Gun Crazy' 'The Endless Summer'
 'Shall We Dance' 'Stalker' 'Aparajito' 'Land of Silence and Darkness'
 'Diabolique' 'The Goat']
Recommendation for user:  609
MovieIds:  ['Richard III' 'Restoration' 'Lawnmower Man 2: Beyond Cyberspace'
 'Belle de Jour' 'Lord of Illusions' 'Burnt by the Sun'
 'The Secret of Roan Inish' 'Virtuosity' 'Wyatt Earp' 'Surviving the Game']
Recommendation for user:  668
MovieIds:  ["Things to Do in Denver When You're Dead" 'The Remains of the Day'
 'The Godfather' 'The Gay Divorcee' 'Big Nig

Recommendation for user:  117
MovieIds:  ['Waiting to Exhale' 'Sabrina' 'Money Train' 'Restoration' 'To Die For'
 'Georgia' 'Friday' 'Kicking and Screaming' 'The White Balloon'
 'Anne Frank Remembered']
Recommendation for user:  169
MovieIds:  ['Braveheart' 'Amateur' 'Priest' 'Forrest Gump' 'Red Rock West'
 'Killing Zoe' 'A Simple Twist of Fate' 'Dead Man'
 'The Adventures of Robin Hood' 'Top Gun']
Recommendation for user:  331
MovieIds:  ['The Last Detail' 'The Outlaw Josey Wales' 'The Endless Summer'
 'Breathless' 'The Lord of the Rings: The Fellowship of the Ring'
 'The Lord of the Rings: The Two Towers' 'Aguirre: The Wrath of God'
 'The Lord of the Rings: The Return of the King' 'Steamboat Bill, Jr.'
 'Holiday']
Recommendation for user:  347
MovieIds:  ['Aguirre: The Wrath of God' 'The Endless Summer' 'Breathless'
 'Dear Frankie' 'Crimes and Misdemeanors' 'El Topo' 'Auntie Mame'
 'Hands on a Hard Body: The Documentary' 'Yojimbo'
 'The Magnificent Ambersons']
Recommendation for user

Recommendation for user:  610
MovieIds:  ["Things to Do in Denver When You're Dead" 'Picnic at Hanging Rock'
 'Women on the Verge of a Nervous Breakdown' 'Titus' 'Benji' 'Frida'
 "Hachi: A Dog's Tale" 'Living in Oblivion' 'Shag' 'Missing']
Recommendation for user:  366
MovieIds:  ['Naked' 'A Walk on the Moon' 'Titus' 'The Dish' 'Breathless' 'Lantana'
 'Frida' 'Heaven Knows, Mr. Allison' 'Dear Frankie' 'El Topo']
Recommendation for user:  397
MovieIds:  ['Titus' 'Breathless' 'Mary and Max' 'The Endless Summer'
 'The Magnificent Ambersons' 'The Help' 'The Kid' 'Holiday'
 'Women on the Verge of a Nervous Breakdown' 'Nowhere in Africa']
Recommendation for user:  542
MovieIds:  ['Balto' "Antonia's Line" 'Blue in the Face' 'My Family'
 'Thirty Two Short Films About Glenn Gould' 'Paris, France'
 'Pather Panchali' 'Dead Man' 'Death in the Garden' 'Cyclo']
Recommendation for user:  6
MovieIds:  ['The Endless Summer' 'Aguirre: The Wrath of God' 'Withnail & I'
 'Breathless' 'Yojimbo' 'Holiday' 'T

## Item-based 

In [32]:
#loading model pickle file
filename2 = 'item-based_CF.pickle'
item_cf = pickle.load(open(filename2, 'rb'))

In [33]:
# Predicting rating for the dataset
pred_rating = item_cf.test(data_testset)

In [34]:
# Saving predictions 
predictions = []
for i in range(len(pred_rating)):
    temp = pred_rating[i].est
    temp1 = round(temp,2)
    predictions.append(temp1)

In [35]:
# Converting to dataframe
predictions_df = pd.DataFrame(predictions, columns=['predictions'])

In [36]:
# Loading predictions onto dataset
item_cf_df = test_data.join(predictions_df)

In [37]:
#Dropping previous 'rating' column
item_cf_df  = item_cf_df.drop(columns=['rating'])

In [38]:
item_cf_df

,userId,movieId,predictions
0,1,1,2.71
1,1,2,2.23
2,1,3,1.89
3,1,4,1.22
4,1,5,2.09
...,...,...,...
5967518,227,161944,5.00
5967519,227,162542,3.54
5967520,227,162672,3.00
5967521,227,163056,3.54


### Printing the recommendations for Item-Based Collaborative Filtering

In [39]:
item_cf_df = pd.merge(item_cf_df, movie_title, how='left', left_on=['movieId'], right_on = ['movieId'])

In [40]:
sorted_pred = item_cf_df.sort_values(by =['userId', 'predictions'], ascending= False)

In [41]:
users = item_cf_df.userId.unique()
for i in range(len(users)):
    top10movies = sorted_pred.loc[(sorted_pred['userId'] == users[i])]['title'].values
    top10movies = top10movies[:10]
    print('Recommendation for user: ', users[i])
    print('MovieIds: ',top10movies)

Recommendation for user:  1
MovieIds:  ['Picture Bride' 'Red Firecracker, Green Firecracker' 'Paris, France'
 'Faces' 'Death in the Garden' "Vive L'Amour" 'Angel Baby'
 'Prisoner of the Mountains' 'Dream with the Fishes' 'Gabbeh']
Recommendation for user:  7
MovieIds:  ['Picture Bride' 'Paris, France' 'Death in the Garden' 'Angel Baby'
 'Prisoner of the Mountains' 'Dream with the Fishes' 'Gabbeh'
 'Storefront Hitchcock' 'The Life of Emile Zola' 'Bandits']
Recommendation for user:  31
MovieIds:  ['Mute Witness' 'Picture Bride' 'Red Firecracker, Green Firecracker'
 'Paris, France' 'Faces' 'Maya Lin: A Strong Clear Vision'
 'Death in the Garden' 'The Day the Sun Turned Cold' "Vive L'Amour"
 'Project S']
Recommendation for user:  32
MovieIds:  ['Lamerica' 'The White Balloon' 'Angela' 'Love & Human Remains'
 'Mute Witness' "L'Enfer" 'Picture Bride'
 'Red Firecracker, Green Firecracker' 'Paris, France'
 'Maya Lin: A Strong Clear Vision']
Recommendation for user:  36
MovieIds:  ['Mute Witness

Recommendation for user:  639
MovieIds:  ['Paris, France' 'Faces' 'Death in the Garden'
 'The Day the Sun Turned Cold' "Vive L'Amour"
 'Message to Love: The Isle of Wight Festival' 'The Life of Emile Zola'
 'Tango' "The Mummy's Ghost" 'House of Frankenstein']
Recommendation for user:  641
MovieIds:  ['Lamerica' 'Angela' 'Love & Human Remains' 'Mute Witness' 'Picture Bride'
 'Red Firecracker, Green Firecracker' 'Paris, France' 'The Promise'
 'Faces' 'Maya Lin: A Strong Clear Vision']
Recommendation for user:  19
MovieIds:  ['Paris, France' 'Faces' 'Death in the Garden' "Vive L'Amour"
 'Prisoner of the Mountains' 'Dream with the Fishes' 'Alien Escape'
 'Storefront Hitchcock' 'Taste of Cherry' 'One Magic Christmas']
Recommendation for user:  35
MovieIds:  ['Faces' "Vive L'Amour" 'Prisoner of the Mountains'
 'Dream with the Fishes' 'Storefront Hitchcock' 'Tango'
 "The Mummy's Ghost" 'House of Frankenstein' 'Son of Frankenstein'
 'The Ghost of Frankenstein']
Recommendation for user:  61
Mov

Recommendation for user:  233
MovieIds:  ["L'Enfer" 'Red Firecracker, Green Firecracker' 'Paris, France'
 'Death in the Garden' 'The Day the Sun Turned Cold' "Vive L'Amour"
 'Project S' 'Little Lord Fauntleroy' 'Hotel de Love' 'Alien Escape']
Recommendation for user:  252
MovieIds:  ['Lamerica' 'Anne Frank Remembered' 'Mute Witness' "L'Enfer"
 'Picture Bride' 'Red Firecracker, Green Firecracker' 'Paris, France'
 'Maya Lin: A Strong Clear Vision' 'Heavy' 'Death in the Garden']
Recommendation for user:  292
MovieIds:  ['Lamerica' 'Anne Frank Remembered' 'Love & Human Remains' 'Mute Witness'
 'Picture Bride' 'Red Firecracker, Green Firecracker' 'Paris, France'
 'Faces' 'Maya Lin: A Strong Clear Vision' 'Heavy']
Recommendation for user:  306
MovieIds:  ['Lamerica' 'Picture Bride' 'Red Firecracker, Green Firecracker' 'Faces'
 'Maya Lin: A Strong Clear Vision' 'Heavy' 'The Day the Sun Turned Cold'
 "Vive L'Amour" 'Project S' 'Angel Baby']
Recommendation for user:  318
MovieIds:  ["Vive L'Amo

Recommendation for user:  198
MovieIds:  ['Lamerica' 'The Amazing Panda Adventure' 'Mute Witness' 'Three Wishes'
 'Gordy' 'Picture Bride' 'Red Firecracker, Green Firecracker' 'Faces'
 'Maya Lin: A Strong Clear Vision' 'Heavy']
Recommendation for user:  229
MovieIds:  ['Lamerica' 'Anne Frank Remembered' 'Mute Witness' 'Picture Bride'
 'Red Firecracker, Green Firecracker' 'Colonel Chabert' 'Paris, France'
 'Faces' 'Maya Lin: A Strong Clear Vision' 'Heavy']
Recommendation for user:  232
MovieIds:  ['Lamerica' 'Anne Frank Remembered' 'Love & Human Remains' 'Mute Witness'
 "L'Enfer" 'Picture Bride' 'Red Firecracker, Green Firecracker'
 'Double Happiness' 'The Wonderful, Horrible Life of Leni Riefenstahl'
 'Paris, France']
Recommendation for user:  262
MovieIds:  ['The Brown Bunny' 'The Edge of Heaven' 'Fox and His Friends'
 'Totally Fucked Up' "Prospero's Books" 'The Saddest Music in the World'
 'Walker' 'Wild Zero' 'Cul-de-sac' 'Z Channel: A Magnificent Obsession']
Recommendation for user:

Recommendation for user:  559
MovieIds:  ['The Usual Suspects' 'Lamerica' 'The White Balloon'
 'Anne Frank Remembered' 'Angela' 'Amateur' 'Love & Human Remains'
 'Mute Witness' 'Nadja' 'Smoke']
Recommendation for user:  598
MovieIds:  ['Lamerica' 'Mute Witness' 'Picture Bride'
 'Red Firecracker, Green Firecracker' 'Paris, France' 'Faces'
 'Maya Lin: A Strong Clear Vision' 'Heavy' 'Death in the Garden'
 'The Day the Sun Turned Cold']
Recommendation for user:  20
MovieIds:  ['Paris, France' 'Death in the Garden' 'Defying Gravity' 'Big Eden'
 'On the Edge' 'I Think I Do' 'Clockstoppers' 'Mystery Date' 'The Trip'
 'Friends and Family']
Recommendation for user:  48
MovieIds:  ['Picture Bride' "Vive L'Amour" 'Angel Baby' 'Gabbeh' 'Alien Escape'
 'And the Ship Sails On' 'Cool as Ice' 'Up at the Villa'
 'The Slipper and the Rose' 'Lured']
Recommendation for user:  78
MovieIds:  ['Lamerica' 'The Postman' 'The White Balloon' 'Angels and Insects'
 'Anne Frank Remembered' 'Chungking Express' 'Amat

...

Recommendation for user:  343
MovieIds:  ['Lamerica' 'Angela' 'Mute Witness' 'Picture Bride'
 'Red Firecracker, Green Firecracker' 'Faces'
 'Maya Lin: A Strong Clear Vision' 'The Day the Sun Turned Cold'
 "Vive L'Amour" 'Project S']
Recommendation for user:  406
MovieIds:  ['Picture Bride' 'Colonel Chabert' 'Faces' 'Heavy' "Vive L'Amour"
 'Little Lord Fauntleroy' 'Angel Baby' 'Hotel de Love' 'Gabbeh'
 'Taste of Cherry']
Recommendation for user:  451
MovieIds:  ['Paris, France' 'Faces' 'Death in the Garden'
 'The Day the Sun Turned Cold' "Vive L'Amour" 'Project S'
 'Little Lord Fauntleroy' 'Message to Love: The Isle of Wight Festival'
 'Prisoner of the Mountains' 'Hotel de Love']
Recommendation for user:  173
MovieIds:  ['Lamerica' 'The White Balloon' 'Anne Frank Remembered'
 'Love & Human Remains' 'Mute Witness' 'Nadja' 'Beyond Bedlam'
 'Picture Bride' 'Red Firecracker, Green Firecracker'
 'The Shawshank Redemption']
Recommendation for user:  223
MovieIds:  ['Lamerica' 'The White Ballo

MovieIds:  ['Lamerica' 'The White Balloon' 'Mute Witness' 'Three Wishes' 'Gordy'
 "L'Enfer" 'Picture Bride' 'Red Firecracker, Green Firecracker'
 'Paris, France' 'Café au Lait']
Recommendation for user:  317
MovieIds:  ['Lamerica' 'Angela' 'Picture Bride' 'Paris, France' 'Death in the Garden'
 'Project S' 'Angel Baby' 'Gabbeh' 'Taste of Cherry' 'Tango']
Recommendation for user:  365
MovieIds:  ['Lamerica' 'The White Balloon' 'Anne Frank Remembered' 'Angela' 'Jeffrey'
 'Love & Human Remains' 'Mute Witness' 'Nadja' 'The Tie That Binds'
 'Little Odessa']
Recommendation for user:  402
MovieIds:  ['Lamerica' 'Mute Witness' 'Picture Bride'
 'Red Firecracker, Green Firecracker' 'Paris, France'
 'Maya Lin: A Strong Clear Vision' 'Heavy' 'Death in the Garden'
 'Little Lord Fauntleroy' 'Female Perversions']
Recommendation for user:  437
MovieIds:  ['Lamerica' 'The White Balloon' 'Mute Witness' "L'Enfer" 'Picture Bride'
 'Red Firecracker, Green Firecracker' 'Paris, France' 'Faces'
 'Maya Lin: A S

Recommendation for user:  609
MovieIds:  ['City of Industry' 'Hav Plenty' 'West Beyrouth' 'And the Ship Sails On'
 'Cool as Ice' 'Defying Gravity' 'The Art of War' 'Under Suspicion'
 'The Circle' 'Big Eden']
Recommendation for user:  668
MovieIds:  ['Lamerica' 'The White Balloon' 'Anne Frank Remembered'
 'The Amazing Panda Adventure' 'Love & Human Remains' 'Mute Witness'
 'Picture Bride' 'Red Firecracker, Green Firecracker' 'Paris, France'
 'Land and Freedom']
Recommendation for user:  210
MovieIds:  ['Prisoner of the Mountains' 'Dream with the Fishes'
 'Storefront Hitchcock' "The Mummy's Ghost" 'House of Frankenstein'
 'Son of Frankenstein' 'The Ghost of Frankenstein' 'Up at the Villa'
 'Lured' 'Two Ninas']
Recommendation for user:  158
MovieIds:  ['Lamerica' 'Picture Bride' 'Paris, France' 'Heavy' 'Death in the Garden'
 'The Day the Sun Turned Cold' "Vive L'Amour" 'Project S'
 'Little Lord Fauntleroy' 'Angel Baby']
Recommendation for user:  127
MovieIds:  ['Othello' 'The Usual Suspec

Recommendation for user:  106
MovieIds:  ['Lamerica' 'The White Balloon' 'Anne Frank Remembered' 'Angela'
 'Mute Witness' 'Nadja' 'Picture Bride'
 'Red Firecracker, Green Firecracker' 'Double Happiness' 'Paris, France']
Recommendation for user:  209
MovieIds:  ['Picture Bride' 'Faces' "Vive L'Amour" 'Angel Baby'
 'Prisoner of the Mountains' 'Dream with the Fishes' 'Gabbeh'
 'Alien Escape' 'Storefront Hitchcock' 'Bandit Queen']
Recommendation for user:  332
MovieIds:  ['The White Balloon' 'Beyond Bedlam' 'Picture Bride'
 'Red Firecracker, Green Firecracker' 'Paris, France' 'Heavy'
 'Death in the Garden' 'The Day the Sun Turned Cold' "Vive L'Amour"
 'The Substance of Fire']
Recommendation for user:  581
MovieIds:  ['Lamerica' 'Feast of July' 'Three Wishes' 'Gordy' 'Picture Bride'
 'Colonel Chabert' 'Paris, France' 'Heavy' 'Death in the Garden'
 'Little Lord Fauntleroy']
Recommendation for user:  296
MovieIds:  ['Cry, the Beloved Country' 'Lamerica' 'Anne Frank Remembered'
 'The Amazing P

Recommendation for user:  204
MovieIds:  ['Paris, France' 'Heavy' 'Death in the Garden' 'Prisoner of the Mountains'
 'Dream with the Fishes' 'Storefront Hitchcock' 'Bandits'
 "The Mummy's Ghost" 'House of Frankenstein' 'Son of Frankenstein']
Recommendation for user:  221
MovieIds:  ['Picture Bride' 'Paris, France' 'Faces' 'Heavy' 'Death in the Garden'
 'The Day the Sun Turned Cold' "Vive L'Amour" 'Angel Baby'
 'Prisoner of the Mountains' 'Dream with the Fishes']
Recommendation for user:  45
MovieIds:  ['Lamerica' 'Angela' 'Mute Witness' 'Red Firecracker, Green Firecracker'
 'Paris, France' 'Land and Freedom' 'Heavy' 'Death in the Garden'
 'The Day the Sun Turned Cold' "Vive L'Amour"]
Recommendation for user:  289
MovieIds:  ['The White Balloon' 'Anne Frank Remembered' 'The Amazing Panda Adventure'
 'Love & Human Remains' 'Mute Witness' 'Picture Bride' 'Paris, France'
 'Pather Panchali' 'Faces' 'Heavy']
Recommendation for user:  444
MovieIds:  ['Lamerica' 'Mute Witness' 'Red Firecracker

### User-based

In [42]:
#loading model pickle file
filename3 = 'user-based_CF.pickle'
item_cf = pickle.load(open(filename3, 'rb'))

In [43]:
# Predicting rating for the dataset
pred_rating = item_cf.test(data_testset)

In [44]:
# Saving predictions 
predictions = []
for i in range(len(pred_rating)):
    temp = pred_rating[i].est
    temp1 = round(temp,2)
    predictions.append(temp1)

In [45]:
# Converting to dataframe
predictions_df = pd.DataFrame(predictions, columns=['predictions'])

In [46]:
# Loading predictions onto dataset
user_cf_df = test_data.join(predictions_df)

In [47]:
#Dropping previous 'rating' column
user_cf_df  = user_cf_df.drop(columns=['rating'])

In [48]:
user_cf_df

,userId,movieId,predictions
0,1,1,2.55
1,1,2,2.05
2,1,3,1.77
3,1,4,1.00
4,1,5,1.93
...,...,...,...
5967518,227,161944,4.14
5967519,227,162542,3.54
5967520,227,162672,3.67
5967521,227,163056,3.54


### Printing the recommendations for User-Based Collaborative Filtering

In [49]:
user_cf_df = pd.merge(user_cf_df, movie_title, how='left', left_on=['movieId'], right_on = ['movieId'])

In [50]:
sorted_pred = user_cf_df.sort_values(by =['userId', 'predictions'], ascending= False)

In [51]:
users = user_cf_df.userId.unique()
for i in range(len(users)):
    top10movies = sorted_pred.loc[(sorted_pred['userId'] == users[i])]['title'].values
    top10movies = top10movies[:10]
    print('Recommendation for user: ', users[i])
    print('MovieIds: ',top10movies)

Recommendation for user:  1
MovieIds:  ['Vampyros Lesbos' 'Dylan Moran: Monster' 'The King Is Alive' 'Innocence'
 'Maelström' 'Incendies' 'Cool as Ice'
 'The Looney, Looney, Looney Bugs Bunny Movie' 'Love Me If You Dare'
 'Seve']
Recommendation for user:  7
MovieIds:  ['Cronos' 'The Day the Sun Turned Cold' 'Prisoner of the Mountains'
 'Dream with the Fishes' 'Storefront Hitchcock' 'Taste of Cherry'
 'Vampyros Lesbos' 'Cool as Ice' 'The Art of War' 'Two Ninas']
Recommendation for user:  31
MovieIds:  ['Lamerica' 'The White Balloon' 'Anne Frank Remembered' 'Mute Witness'
 'Nadja' 'Picture Bride' 'Red Firecracker, Green Firecracker'
 'The Wonderful, Horrible Life of Leni Riefenstahl'
 'Faster, Pussycat! Kill! Kill!' 'Paris, France']
Recommendation for user:  32
MovieIds:  ['Lamerica' 'Mute Witness' 'Picture Bride' 'Cronos' 'Heavy'
 'The Day the Sun Turned Cold' 'Project S' 'Female Perversions'
 'Angel Baby' 'Prisoner of the Mountains']
Recommendation for user:  36
MovieIds:  ['Mute Witne

Recommendation for user:  35
MovieIds:  ['Death of a Superhero' 'Vampyros Lesbos' 'Dylan Moran: Monster'
 'Excision' 'The King Is Alive' 'Innocence' 'Maelström' 'Incendies'
 'Cool as Ice' 'The Looney, Looney, Looney Bugs Bunny Movie']
Recommendation for user:  61
MovieIds:  ['Cronos' 'The Day the Sun Turned Cold' 'Prisoner of the Mountains'
 'Dream with the Fishes' 'Storefront Hitchcock' 'Taste of Cherry'
 'Vampyros Lesbos' 'Cool as Ice' 'The Art of War' 'Two Ninas']
Recommendation for user:  99
MovieIds:  ['Picture Bride' 'Cronos' 'The Day the Sun Turned Cold' 'Angel Baby'
 'Prisoner of the Mountains' 'Dream with the Fishes' 'Gabbeh'
 'Storefront Hitchcock' 'Taste of Cherry' 'Vampyros Lesbos']
Recommendation for user:  102
MovieIds:  ['Lamerica' 'Anne Frank Remembered' 'Mute Witness' 'Picture Bride'
 'Red Firecracker, Green Firecracker' 'Paris, France' 'Cronos'
 'Maya Lin: A Strong Clear Vision' 'Heavy' 'Walking and Talking']
Recommendation for user:  128
MovieIds:  ['Lamerica' 'Anne 

Recommendation for user:  380
MovieIds:  ['The Day the Sun Turned Cold' 'Prisoner of the Mountains'
 'Dream with the Fishes' 'Storefront Hitchcock' 'Taste of Cherry'
 'Vampyros Lesbos' 'Cool as Ice' 'The Art of War' 'Two Ninas'
 'The King Is Alive']
Recommendation for user:  507
MovieIds:  ['Cronos' 'The Day the Sun Turned Cold' 'Prisoner of the Mountains'
 'Dream with the Fishes' 'Storefront Hitchcock' 'Taste of Cherry'
 'Vampyros Lesbos' 'Cool as Ice' 'The Art of War' 'Two Ninas']
Recommendation for user:  509
MovieIds:  ['The Day the Sun Turned Cold' 'Prisoner of the Mountains'
 'Dream with the Fishes' 'Storefront Hitchcock' 'Taste of Cherry'
 'Vampyros Lesbos' 'Cool as Ice' 'The Art of War' 'Two Ninas'
 'The King Is Alive']
Recommendation for user:  534
MovieIds:  ['Lamerica' 'Mute Witness' 'Picture Bride' 'Paris, France' 'Cronos'
 'Death in the Garden' 'The Day the Sun Turned Cold' 'Project S'
 'Female Perversions' 'Angel Baby']
Recommendation for user:  537
MovieIds:  ['Lamerica'

Recommendation for user:  198
MovieIds:  ['Lamerica' 'Anne Frank Remembered' 'Mute Witness' 'Picture Bride'
 'Red Firecracker, Green Firecracker' 'Paris, France' 'Cronos'
 'Maya Lin: A Strong Clear Vision' 'Heavy' 'Walking and Talking']
Recommendation for user:  229
MovieIds:  ['Lamerica' 'The White Balloon' 'Anne Frank Remembered'
 'Love & Human Remains' 'Mute Witness' 'Nadja' 'Picture Bride'
 'Red Firecracker, Green Firecracker'
 'The Wonderful, Horrible Life of Leni Riefenstahl'
 'Faster, Pussycat! Kill! Kill!']
Recommendation for user:  232
MovieIds:  ['Lamerica' 'Anne Frank Remembered' 'Mute Witness' 'Picture Bride'
 'Red Firecracker, Green Firecracker' 'Paris, France' 'Cronos'
 'Maya Lin: A Strong Clear Vision' 'Heavy' 'Walking and Talking']
Recommendation for user:  262
MovieIds:  ['Prisoner of the Mountains' 'Dream with the Fishes'
 'Storefront Hitchcock' 'Two Ninas' 'Erik the Viking'
 'The Grass Is Greener' 'Funeral in Berlin' 'Caveman'
 'Death of a Superhero' 'The King Is Ali

Recommendation for user:  313
MovieIds:  ['Picture Bride' 'Cronos' 'The Day the Sun Turned Cold' 'Project S'
 'Angel Baby' 'Prisoner of the Mountains' 'Dream with the Fishes' 'Gabbeh'
 'Storefront Hitchcock' 'Taste of Cherry']
Recommendation for user:  326
MovieIds:  ['Picture Bride' 'The Day the Sun Turned Cold' 'Project S' 'Angel Baby'
 'Gabbeh' 'Taste of Cherry' 'Armour of God' 'Vampyros Lesbos'
 'Cool as Ice' 'The Art of War']
Recommendation for user:  335
MovieIds:  ['Mute Witness' 'Cronos' 'The Day the Sun Turned Cold' 'Project S'
 'Female Perversions' 'Prisoner of the Mountains' 'Losing Chase'
 'The Designated Mourner' 'Dream with the Fishes' 'Phantoms']
Recommendation for user:  388
MovieIds:  ['Lamerica' 'Mute Witness' 'Picture Bride' 'Paris, France' 'Cronos'
 'Death in the Garden' 'The Day the Sun Turned Cold' 'Project S'
 'Female Perversions' 'Angel Baby']
Recommendation for user:  404
MovieIds:  ['Lamerica' 'Mute Witness' 'Picture Bride'
 'Red Firecracker, Green Firecracker

...

Recommendation for user:  378
MovieIds:  ['The Day the Sun Turned Cold' 'Prisoner of the Mountains'
 'Dream with the Fishes' 'Storefront Hitchcock' 'Vampyros Lesbos'
 'Cool as Ice' 'The Art of War' 'Two Ninas' 'The King Is Alive'
 'Erik the Viking']
Recommendation for user:  425
MovieIds:  ['Cronos' 'The Day the Sun Turned Cold' 'Prisoner of the Mountains'
 'Dream with the Fishes' 'Storefront Hitchcock' 'Taste of Cherry'
 'Vampyros Lesbos' 'Cool as Ice' 'The Art of War' 'Two Ninas']
Recommendation for user:  438
MovieIds:  ['Lamerica' 'The White Balloon' 'Last Summer in the Hamptons'
 'Anne Frank Remembered' 'The Amazing Panda Adventure'
 'Love & Human Remains' 'Mute Witness' 'Nadja' 'Smoke' 'Burnt by the Sun']
Recommendation for user:  454
MovieIds:  ['Lamerica' 'The White Balloon' 'Last Summer in the Hamptons' 'La Haine'
 'Anne Frank Remembered' 'Chungking Express' 'Angela'
 'The Amazing Panda Adventure' 'An Awfully Big Adventure' 'Amateur']
Recommendation for user:  483
MovieIds:  [

Recommendation for user:  538
MovieIds:  ['Lamerica' 'Mute Witness' 'Picture Bride' 'Heavy'
 'The Day the Sun Turned Cold' 'Project S' 'Female Perversions'
 'Angel Baby' 'Losing Chase' 'The Designated Mourner']
Recommendation for user:  570
MovieIds:  ['Lamerica' 'Mute Witness' 'Picture Bride' 'Paris, France' 'Cronos'
 'Maya Lin: A Strong Clear Vision' 'Heavy' 'Death in the Garden'
 'The Day the Sun Turned Cold' 'Project S']
Recommendation for user:  623
MovieIds:  ['Lamerica' 'The White Balloon' 'Anne Frank Remembered' 'Mute Witness'
 'Nadja' 'Picture Bride' 'Red Firecracker, Green Firecracker'
 'Faster, Pussycat! Kill! Kill!' 'Cronos'
 'Maya Lin: A Strong Clear Vision']
Recommendation for user:  628
MovieIds:  ['Vampyros Lesbos' 'Cool as Ice' 'The Art of War' 'The King Is Alive'
 'Innocence' 'Maelström' 'The Looney, Looney, Looney Bugs Bunny Movie'
 'Land of Silence and Darkness' 'The Goat' 'The Play House']
Recommendation for user:  629
MovieIds:  ['Cronos' 'The Day the Sun Turned C

Recommendation for user:  368
MovieIds:  ['Lamerica' 'Anne Frank Remembered' 'Mute Witness' 'Picture Bride'
 'Red Firecracker, Green Firecracker' 'Cronos'
 'Maya Lin: A Strong Clear Vision' 'Heavy' 'Walking and Talking'
 'The Day the Sun Turned Cold']
Recommendation for user:  645
MovieIds:  ['Lamerica' 'Mute Witness' 'Red Firecracker, Green Firecracker'
 'Double Happiness' 'My Favorite Season' 'Maya Lin: A Strong Clear Vision'
 'Heavy' 'The Day the Sun Turned Cold' 'Project S' 'Female Perversions']
Recommendation for user:  515
MovieIds:  ['The Usual Suspects' 'Lamerica' 'The Confessional' 'The White Balloon'
 'La Haine' 'Unforgettable' 'Anne Frank Remembered'
 'The Amazing Panda Adventure' 'Amateur' 'Mute Witness']
Recommendation for user:  107
MovieIds:  ['Vampyros Lesbos' 'The Art of War' 'The King Is Alive' 'Innocence'
 'Maelström' 'Dylan Moran: Monster' 'Death of a Superhero' 'Cool as Ice'
 'The Looney, Looney, Looney Bugs Bunny Movie' 'Seve']
Recommendation for user:  112
MovieI

Recommendation for user:  469
MovieIds:  ['Mute Witness' 'Nadja' 'Red Firecracker, Green Firecracker'
 'Maya Lin: A Strong Clear Vision' 'Heavy' 'Walking and Talking'
 'Project S' 'A Perfect Candidate' 'Female Perversions' 'Hearts and Minds']
Recommendation for user:  115
MovieIds:  ['Lamerica' 'The White Balloon' 'Anne Frank Remembered'
 'The Amazing Panda Adventure' 'Mute Witness' 'Nadja' 'Picture Bride'
 'Red Firecracker, Green Firecracker'
 'The Wonderful, Horrible Life of Leni Riefenstahl' 'Paris, France']
Recommendation for user:  272
MovieIds:  ['Lamerica' 'Mute Witness' 'Picture Bride' 'Double Happiness' 'Cronos'
 'Maya Lin: A Strong Clear Vision' 'Heavy' 'The Day the Sun Turned Cold'
 'Project S' 'Female Perversions']
Recommendation for user:  349
MovieIds:  ['Mute Witness' 'Nadja' 'Picture Bride' 'Double Happiness' 'Cronos'
 'The Day the Sun Turned Cold' 'Project S' 'Female Perversions'
 'Angel Baby' 'Prisoner of the Mountains']
Recommendation for user:  489
MovieIds:  ['Lame

Recommendation for user:  504
MovieIds:  ['Vampyros Lesbos' 'Cool as Ice' 'The King Is Alive' 'Innocence'
 'Maelström' 'The Looney, Looney, Looney Bugs Bunny Movie'
 'Dylan Moran: Monster' 'Seve' 'Land of Silence and Darkness' 'The Goat']
Recommendation for user:  301
MovieIds:  ['Unforgettable' 'Amateur' 'Mute Witness' 'Nadja' 'Hideaway'
 'Double Happiness' 'My Favorite Season' 'The Day the Sun Turned Cold'
 'Female Perversions' 'That Old Feeling']
Recommendation for user:  123
MovieIds:  ['Lamerica' 'Mute Witness' 'Red Firecracker, Green Firecracker'
 'Maya Lin: A Strong Clear Vision' 'Heavy' 'Walking and Talking'
 'The Day the Sun Turned Cold' 'Project S' 'Female Perversions'
 'Losing Chase']
Recommendation for user:  258
MovieIds:  ['Lamerica' 'The White Balloon' 'Anne Frank Remembered'
 'Love & Human Remains' 'Mute Witness' 'Nadja' 'Picture Bride'
 'Red Firecracker, Green Firecracker'
 'The Wonderful, Horrible Life of Leni Riefenstahl'
 'Faster, Pussycat! Kill! Kill!']
Recommendat

# Hybrid Models

# Hybrid Approach A (Content-Based and Item-Based Collaborative Filter)

In [52]:

df = pd.DataFrame()
df = pd.merge(item_cf_df, recom_content_pred, how= 'inner', left_on=['userId','movieId'], right_on = ['userId','movieId'])

In [53]:
df_test = pd.DataFrame()
df_test['predictions'] = df['predictions']
df_test['prediction'] = df['prediction']
x_test = df_test.values

In [54]:
with open('hybrid_collaborative-item_content.pkl', 'rb') as file:
    pickle_model = pickle.load(file)
    
Ypredict = pickle_model.predict(x_test)

In [55]:
df['hybrid_pred'] =  Ypredict 

In [56]:
df.head()

,userId,movieId,predictions,title_x,prediction,title_y,hybrid_pred
0,1,1,2.71,Toy Story,3.539915,Toy Story,3.698917
1,1,2,2.23,Jumanji,3.539915,Jumanji,3.698917
2,1,3,1.89,Grumpier Old Men,3.539915,Grumpier Old Men,3.698917
3,1,4,1.22,Waiting to Exhale,3.539915,Waiting to Exhale,3.698917
4,1,5,2.09,Father of the Bride Part II,3.539915,Father of the Bride Part II,3.698917


### Printing the recommendations for Hybrid Approach A

In [57]:
df= pd.merge(df, movie_title, how='left', left_on=['movieId'], right_on = ['movieId'])

In [58]:
sorted_pred = df.sort_values(by =['userId', 'hybrid_pred'], ascending= False)

In [59]:
users = df.userId.unique()
for i in range(len(users)):
    top10movies = sorted_pred.loc[(sorted_pred['userId'] == users[i])]['title'].values
    top10movies = top10movies[:10]
    print('Recommendation for user: ', users[i])
    print('MovieIds: ',top10movies)

Recommendation for user:  1
MovieIds:  ['Toy Story' 'Jumanji' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Heat' 'Sabrina' 'Tom and Huck'
 'Sudden Death' 'GoldenEye']
Recommendation for user:  7
MovieIds:  ['Jumanji' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Heat' 'Sabrina' 'Tom and Huck'
 'Sudden Death' 'The American President' 'Dracula: Dead and Loving It']
Recommendation for user:  31
MovieIds:  ['Jason and the Argonauts' 'My Super Ex-Girlfriend'
 'The Bridges of Madison County' 'Jack & Sarah' 'Pollyanna'
 'Blood, Guts, Bullets and Octane' 'To Sir, with Love' 'Elephant'
 'Only the Strong' 'Born Rich']
Recommendation for user:  32
MovieIds:  ['The Paleface' 'Pathfinder' 'Ong-Bak: The Thai Warrior' 'Toy Story'
 'Grumpier Old Men' 'Waiting to Exhale' 'Father of the Bride Part II'
 'Heat' 'Sabrina' 'Tom and Huck']
Recommendation for user:  36
MovieIds:  ['Star Trek V: The Final Frontier' 'Roadgames' 'İtirazım Var' 'Toy Story'
 'Juma

Recommendation for user:  641
MovieIds:  ['Mask' 'Malice' 'Rebel Without a Cause' 'Condorman' 'Dirty Work'
 'Indecent Proposal' 'Radio Days' "White Men Can't Jump" 'Bound for Glory'
 "Max Keeble's Big Move"]
Recommendation for user:  19
MovieIds:  ['Mask' "The Ballad of Ramblin' Jack" 'The Thin Red Line'
 'Goodbye, Columbus' 'Rudolph, the Red-Nosed Reindeer' 'A Midnight Clear'
 'Father of the Bride Part II' 'Tom and Huck'
 'Dracula: Dead and Loving It' 'Balto']
Recommendation for user:  35
MovieIds:  ['Toy Story' 'Jumanji' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Heat' 'Sabrina' 'Tom and Huck'
 'Sudden Death' 'GoldenEye']
Recommendation for user:  61
MovieIds:  ['Zathura: A Space Adventure' 'Toy Story' 'Grumpier Old Men'
 'Waiting to Exhale' 'Father of the Bride Part II' 'Heat' 'Sabrina'
 'Tom and Huck' 'Sudden Death' 'GoldenEye']
Recommendation for user:  99
MovieIds:  ['Indecent Proposal' 'A Midnight Clear' 'The Messenger' 'Malice'
 'Condorman' 'Radio Da

Recommendation for user:  233
MovieIds:  ['Arthur' 'Toy Story' 'Jumanji' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Heat' 'Tom and Huck' 'Sudden Death'
 'GoldenEye' 'The American President']
Recommendation for user:  252
MovieIds:  ["White Men Can't Jump" 'Jumanji' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Tom and Huck' 'Sudden Death' 'GoldenEye'
 'The American President' 'Balto']
Recommendation for user:  292
MovieIds:  ['Dirty Work' 'Indecent Proposal' 'The Thin Red Line' 'Let It Ride'
 'Goodbye, Columbus' 'Mask' 'Rudolph, the Red-Nosed Reindeer' 'Room'
 'Match Point' 'Malice']
Recommendation for user:  306
MovieIds:  ['The Thin Red Line' 'A Midnight Clear' 'Mask' 'Tai-Chi Master'
 'Stanley Kubrick: A Life in Pictures' 'Match Point' 'Grumpier Old Men'
 'Waiting to Exhale' 'Heat' 'Tom and Huck']
Recommendation for user:  318
MovieIds:  ['Jumanji' 'Grumpier Old Men' 'Waiting to Exhale' 'Sabrina' 'Tom and Huck'
 'GoldenEye' 'The American President' 

Recommendation for user:  229
MovieIds:  ['Casualties of War' 'Everything is Illuminated' 'Toy Story' 'Jumanji'
 'Grumpier Old Men' 'Waiting to Exhale' 'Father of the Bride Part II'
 'Heat' 'Sabrina' 'Tom and Huck']
Recommendation for user:  232
MovieIds:  ['Indecent Proposal' "Harry Potter and the Philosopher's Stone"
 'Stanley Kubrick: A Life in Pictures' 'Match Point'
 "White Men Can't Jump" 'A Midnight Clear' 'Let It Ride' 'Mask'
 'The Messenger' 'Hollywood Ending']
Recommendation for user:  262
MovieIds:  ['Room' 'The Messenger' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Heat' 'Sabrina' 'Tom and Huck'
 'Sudden Death' 'GoldenEye']
Recommendation for user:  280
MovieIds:  ['Father of the Bride Part II' 'When a Man Loves a Woman'
 'Eyes Without a Face' 'Meet Me in St. Louis' 'Reservoir Dogs'
 'A Thousand Acres' 'Blame It on Rio' 'What Dreams May Come'
 'Hideous Kinky' 'The Tomb of Ligeia']
Recommendation for user:  283
MovieIds:  ['The Thin Red Line' 'Maid

MovieIds:  ['Maid in Manhattan' 'Jumanji' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Heat' 'Sabrina' 'Tom and Huck'
 'Sudden Death' 'GoldenEye']
Recommendation for user:  488
MovieIds:  ['Kiss the Girls' 'Fallen' 'Sabotage' 'The Bone Collector' 'The Cell'
 'Along Came a Spider' 'Sudden Impact' 'Murder by Numbers' 'Insomnia'
 'The Ring']
Recommendation for user:  559
MovieIds:  ['Indecent Proposal' 'The Thin Red Line' 'The Funhouse' 'Let It Ride'
 'A Midnight Clear' 'Goodbye, Columbus' 'Malice' 'Rebel Without a Cause'
 'Condorman' 'Radio Days']
Recommendation for user:  598
MovieIds:  ['Radio Days' 'The Funhouse' "White Men Can't Jump" 'Bound for Glory'
 'Hollywood Ending' "What's Up, Tiger Lily?" 'For Your Consideration'
 'The Messenger' 'Africa Screams' 'Rebel Without a Cause']
Recommendation for user:  20
MovieIds:  ['Bringing Up Baby' 'Dirty Work' 'The Funhouse' 'Jumanji'
 'Grumpier Old Men' 'Waiting to Exhale' 'Father of the Bride Part II'
 'Heat' 'Sabri

...

Recommendation for user:  483
MovieIds:  ['Blood and Wine' 'Bob le Flambeur' 'The Getaway' 'White Heat'
 "Henry's Crime" 'Deadfall' 'Odds Against Tomorrow'
 'Like Water for Chocolate' 'The Sting' 'Still Breathing']
Recommendation for user:  494
MovieIds:  ['Like Water for Chocolate' 'Daylight' 'Still Breathing'
 'The Dangerous Lives of Altar Boys' 'Odds Against Tomorrow'
 'Throne of Blood' 'White Heat' 'The Return' 'Four Lions' "Henry's Crime"]
Recommendation for user:  499
MovieIds:  ['Like Water for Chocolate' 'Daylight' 'Still Breathing'
 'The Dangerous Lives of Altar Boys' 'Odds Against Tomorrow'
 'Throne of Blood' 'Four Lions' 'Hotel Chevalier' 'Nightcrawler'
 'Toy Story']
Recommendation for user:  503
MovieIds:  ['Malice' 'Rebel Without a Cause' 'Condorman' 'Indecent Proposal'
 'The Thin Red Line' 'Radio Days' 'The Funhouse' 'Toy Story 2'
 "White Men Can't Jump" 'Bound for Glory']
Recommendation for user:  523
MovieIds:  ['Toy Story' 'Jumanji' 'Grumpier Old Men' 'Waiting to Exhal

Recommendation for user:  473
MovieIds:  ['Maverick' 'D3: The Mighty Ducks' 'Peter Pan' 'eXistenZ'
 'Any Given Sunday' 'Dungeons & Dragons' 'Manhunter' 'Panic Room'
 'Rollerball' 'Spy Kids 3-D: Game Over']
Recommendation for user:  513
MovieIds:  ['A Close Shave' 'Stalingrad' 'Henry V' 'Breaking the Waves'
 'The Last of the Mohicans' 'Mrs. Miniver' 'Tea with Mussolini'
 'Village of the Damned' 'The Lords of Flatbush' 'Scandal']
Recommendation for user:  538
MovieIds:  ['Malice' 'Rebel Without a Cause' 'Condorman' 'Indecent Proposal'
 'Radio Days' 'The Funhouse' 'Toy Story 2' "White Men Can't Jump"
 'Bound for Glory' 'Hollywood Ending']
Recommendation for user:  570
MovieIds:  ['Jumanji' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Heat' 'Sabrina' 'Tom and Huck'
 'Sudden Death' 'GoldenEye' 'The American President']
Recommendation for user:  623
MovieIds:  ['Malice' 'Rebel Without a Cause' 'Condorman' 'Indecent Proposal'
 'The Thin Red Line' 'Radio Days' 'The Fu

Recommendation for user:  644
MovieIds:  ['Edge of Seventeen' 'Worth Winning' 'The Postman Always Rings Twice'
 'The Wizard of Oz' 'Toy Story' 'Jumanji' 'Grumpier Old Men'
 'Father of the Bride Part II' 'Heat' 'Sabrina']
Recommendation for user:  12
MovieIds:  ['Deep Red' 'Toy Story' 'Jumanji' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Heat' 'Sabrina' 'Tom and Huck'
 'Sudden Death']
Recommendation for user:  189
MovieIds:  ['American Flyers' 'Band of the Hand' 'Cocaine Cowboys' 'Point Blank'
 'Toy Story' 'Jumanji' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Heat']
Recommendation for user:  215
MovieIds:  ['The Funhouse' 'The Thin Red Line' 'Let It Ride' 'Goodbye, Columbus'
 'Africa Screams' 'Jumanji' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Sabrina']
Recommendation for user:  368
MovieIds:  ['The Swan Princess' 'Princess Caraboo' 'The Empire Strikes Back'
 'The Princess Bride' 'Return of the Jedi' 'His

Recommendation for user:  101
MovieIds:  ["Charlie's Angels: Full Throttle" 'Toy Story' 'Jumanji'
 'Grumpier Old Men' 'Waiting to Exhale' 'Father of the Bride Part II'
 'Heat' 'Sabrina' 'Tom and Huck' 'Sudden Death']
Recommendation for user:  591
MovieIds:  ['Toy Story' 'Jumanji' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Heat' 'Sabrina' 'Tom and Huck'
 'Sudden Death' 'GoldenEye']
Recommendation for user:  156
MovieIds:  ['Vertigo' 'Bad Moon' 'Charly' 'Battle Cry' "It's All About Love"
 'Super Fly' 'The Jerk' "Mo' Better Blues" 'Urban Cowboy'
 'The Girl Who Leapt Through Time']
Recommendation for user:  642
MovieIds:  ['Mary Reilly' 'Backbeat' 'The Edge' "Who's Harry Crumb?"
 'Titicut Follies' 'Inception' 'Ai Weiwei: Never Sorry' 'Elizabethtown'
 'Interstellar' 'Toy Story']
Recommendation for user:  469
MovieIds:  ['Disclosure' 'Outbreak' '2001: A Space Odyssey' 'The Lawnmower Man'
 'Sneakers' 'Speed 2: Cruise Control' 'The Computer Wore Tennis Shoes'
 'The 

Recommendation for user:  65
MovieIds:  ['The Swan Princess' 'Princess Caraboo' 'The Princess Bride'
 'History of the World: Part I' 'The Mummy' 'The 7th Voyage of Sinbad'
 'Shanghai Noon' 'Sinbad and the Eye of the Tiger'
 "Bill & Ted's Bogus Journey"
 'Spacehunter: Adventures in the Forbidden Zone']
Recommendation for user:  638
MovieIds:  ['Enter the Dragon' 'Toy Story' 'Jumanji' 'Grumpier Old Men'
 'Waiting to Exhale' 'Father of the Bride Part II' 'Heat' 'Sabrina'
 'Tom and Huck' 'Sudden Death']
Recommendation for user:  14
MovieIds:  ['Toy Story' 'Jumanji' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Heat' 'Sabrina' 'Tom and Huck'
 'Sudden Death' 'GoldenEye']
Recommendation for user:  269
MovieIds:  ['Toy Story' 'Jumanji' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Heat' 'Sabrina' 'Tom and Huck'
 'Sudden Death' 'GoldenEye']
Recommendation for user:  322
MovieIds:  ['Inherent Vice' 'The White Stripes: Under Great White Northern L

# Hybrid Approach B (Content-Based and Model-Based Collaborative Filter)

In [60]:
df = pd.DataFrame()
df = pd.merge(model_cf_df, recom_content_pred, how= 'inner', left_on=['userId','movieId'], right_on = ['userId','movieId'])

In [61]:
df_test = pd.DataFrame()
df_test['predictions'] = df['predictions']
df_test['prediction'] = df['prediction']
x_test = df_test.values

In [65]:
#loading model pickle file
with open('hybrid_b.pickle', 'rb') as file:
    pickle_model = pickle.load(file)
    
Ypredict = pickle_model.predict(x_test)

In [66]:
df['hybrid_pred'] =  Ypredict 

In [67]:
df.head()

,userId,movieId,predictions,title_x,prediction,title_y,hybrid_pred
0,1,1,2.72,Toy Story,3.539915,Toy Story,3.484281
1,1,2,2.41,Jumanji,3.539915,Jumanji,3.463903
2,1,3,2.15,Grumpier Old Men,3.539915,Grumpier Old Men,3.446811
3,1,4,1.00,Waiting to Exhale,3.539915,Waiting to Exhale,3.371213
4,1,5,1.98,Father of the Bride Part II,3.539915,Father of the Bride Part II,3.435636


### Printing the recommendations for Hybrid Approach B

In [68]:
df= pd.merge(df, movie_title, how='left', left_on=['movieId'], right_on = ['movieId'])

In [69]:
sorted_pred = df.sort_values(by =['userId', 'hybrid_pred'], ascending= False)

In [70]:
users = df.userId.unique()
for i in range(len(users)):
    top10movies = sorted_pred.loc[(sorted_pred['userId'] == users[i])]['title'].values
    top10movies = top10movies[:10]
    print('Recommendation for user: ', users[i])
    print('MovieIds: ',top10movies)

Recommendation for user:  1
MovieIds:  ['Breathless' 'El Topo' 'Aguirre: The Wrath of God' 'The Endless Summer'
 'Titus' 'Blue in the Face' 'A Walk on the Moon' 'Cyclo' 'Out of the Past'
 'Fitzcarraldo']
Recommendation for user:  7
MovieIds:  ['Land of Silence and Darkness' 'The Play House' 'The Goat' 'Cops'
 'The Last Detail' 'Holiday' 'Steamboat Willie' 'The Endless Summer'
 'Sherlock, Jr.' 'The Godfather']
Recommendation for user:  31
MovieIds:  ['My Super Ex-Girlfriend' 'Jason and the Argonauts' 'Creed' 'The Drop'
 'Suicide Club' 'To Sir, with Love' 'Born Rich' 'Jack & Sarah' 'Pollyanna'
 'Sex Ed']
Recommendation for user:  32
MovieIds:  ['The Paleface' 'Pathfinder' 'Ong-Bak: The Thai Warrior' 'Killing Zoe'
 'Dead Man' 'The Ruling Class' 'Local Hero' 'Sliding Doors'
 'Wild Man Blues' 'And the Band Played On']
Recommendation for user:  36
MovieIds:  ['İtirazım Var' 'Star Trek V: The Final Frontier' 'Roadgames'
 'Nights of Cabiria' 'Titus' "Hachi: A Dog's Tale" 'Dangerous Beauty'
 'A

Recommendation for user:  61
MovieIds:  ['Zathura: A Space Adventure' 'The Brothers Grimm' 'Balto'
 'Blue in the Face' 'Unzipped' 'Dead Man' 'American Buffalo' 'Cyclo'
 'An Affair to Remember' 'The Adventures of Robin Hood']
Recommendation for user:  99
MovieIds:  ['A Midnight Clear' 'The Messenger' 'Indecent Proposal' 'Mask'
 'Radio Days' "White Men Can't Jump" 'Bound for Glory'
 'For Your Consideration' 'The Funhouse' 'Condorman']
Recommendation for user:  102
MovieIds:  ['American Pop' 'Cinderella' 'Earthquake' 'Night and Fog' 'Into the Woods'
 'Sunshine' 'Napoleon Dynamite' 'The Best of Youth' 'Basket Case'
 'How the West Was Won']
Recommendation for user:  128
MovieIds:  ['Malice' 'For Your Consideration' 'The Messenger'
 'Love! Valour! Compassion!' 'Bound for Glory' 'Rebel Without a Cause'
 'Goodbye, Columbus' 'Let It Ride' 'Rudolph, the Red-Nosed Reindeer'
 'Maid in Manhattan']
Recommendation for user:  175
MovieIds:  ['Toy Story 3' "Max Keeble's Big Move" "The Ballad of Ramblin

Recommendation for user:  380
MovieIds:  ['Match Point' 'Condorman' 'A Midnight Clear' 'Malice'
 "What's Up, Tiger Lily?" 'Rudolph, the Red-Nosed Reindeer'
 'Rebel Without a Cause' 'Let It Ride' 'The Adventures of Robin Hood'
 'Beautiful Thing']
Recommendation for user:  507
MovieIds:  ["White Men Can't Jump" 'Love Is a Many-Splendored Thing'
 "Brewster's Millions" 'Nowhere in Africa' 'Dogville' 'Gladiator 1992'
 'Batman' 'Zero Dark Thirty' 'Nights of Cabiria' 'The Intouchables']
Recommendation for user:  509
MovieIds:  ['Maid in Manhattan' 'Let It Ride' 'Smiles of a Summer Night'
 'A Midnight Clear' 'Room' 'The Old Man and the Sea' 'Lawrence of Arabia'
 'Steamboat Willie' 'The Last Detail' 'The Last Laugh']
Recommendation for user:  534
MovieIds:  ["Harry Potter and the Philosopher's Stone" 'Monsters, Inc.'
 'Bound for Glory' 'Radio Days' 'Hollywood Ending' 'Condorman'
 'For Your Consideration' 'Toy Story 2' 'Toy Story 3' 'Africa Screams']
Recommendation for user:  537
MovieIds:  ['To

Recommendation for user:  280
MovieIds:  ['American Beauty' 'Titus' 'The Twilight Samurai' 'In the Land of Women'
 'Confessions' 'Pride & Prejudice' 'Reservoir Dogs' 'The Descendants'
 'Because I Said So' 'Definitely, Maybe']
Recommendation for user:  283
MovieIds:  ['Stanley Kubrick: A Life in Pictures' 'The Thin Red Line'
 'Maid in Manhattan' 'Goodbye, Columbus' 'Stepmom' 'Les Miserables'
 'Faster, Pussycat! Kill! Kill!' 'The Secret Adventures of Tom Thumb'
 'Cronos' 'Maya Lin: A Strong Clear Vision']
Recommendation for user:  320
MovieIds:  ['You Kill Me' 'Sliding Doors' 'O Lucky Man!' 'The Sting' 'All or Nothing'
 '11:14' 'Gloria' 'Secondhand Lions' 'Moll Flanders' 'Sex and Lucia']
Recommendation for user:  321
MovieIds:  ['Rudolph, the Red-Nosed Reindeer' 'A Midnight Clear'
 "Harry Potter and the Philosopher's Stone" 'Shanghai Triad' 'To Die For'
 'The Postman' 'Friday' 'The Crossing Guard'
 "Things to Do in Denver When You're Dead" 'Heidi Fleiss: Hollywood Madam']
Recommendation 

Recommendation for user:  559
MovieIds:  ['Let It Ride' 'A Midnight Clear' 'The Thin Red Line' 'Goodbye, Columbus'
 'The Funhouse' 'Indecent Proposal' 'Toy Story 3' 'Rebel Without a Cause'
 'Toy Story 2' 'Malice']
Recommendation for user:  598
MovieIds:  ['The Funhouse' "What's Up, Tiger Lily?" 'The Messenger' 'Bound for Glory'
 'Hollywood Ending' "White Men Can't Jump" 'For Your Consideration'
 'Radio Days' 'Africa Screams' 'Smiles of a Summer Night']
Recommendation for user:  20
MovieIds:  ['Bringing Up Baby' 'Dirty Work' 'The Funhouse' 'Africa Screams'
 'For Your Consideration' 'Sabrina' 'Money Train' 'Assassins'
 'It Takes Two' 'Georgia']
Recommendation for user:  48
MovieIds:  ['The Thin Red Line' 'A Midnight Clear' 'Mask' 'Let It Ride' 'Stepmom'
 'The Messenger' 'Bound for Glory' 'Hollywood Ending'
 "What's Up, Tiger Lily?" 'Africa Screams']
Recommendation for user:  78
MovieIds:  ['Rollerball' 'Indie Game: The Movie' 'Zathura: A Space Adventure'
 'Avalon' 'Guardians of the Galax

...

Recommendation for user:  104
MovieIds:  ['Henry V' 'The New World' 'A Close Shave' 'Pride & Prejudice' 'What If'
 'A Matter of Loaf and Death' 'The Private Life of Henry VIII'
 'Corpse Bride' 'Never Let Me Go' 'Village of the Damned']
Recommendation for user:  139
MovieIds:  ['Tea with Mussolini' 'King Ralph' 'Never Let Me Go' 'I Shot Andy Warhol'
 'Them!' "Angela's Ashes" 'The Last Detail' 'The Crimson Pirate'
 "Brewster's Millions" 'The Lord of the Rings: The Fellowship of the Ring']
Recommendation for user:  153
MovieIds:  ['Toy Story 3' 'Toy Story 2' 'Room' 'The Funhouse' 'A Midnight Clear'
 'Radio Days' "The Ballad of Ramblin' Jack" 'Bound for Glory'
 'The Messenger' 'Tai-Chi Master']
Recommendation for user:  163
MovieIds:  ['Goodbye, Columbus' 'The Thin Red Line' 'Dead Man' 'Gigi'
 'Beautiful Thing' 'Shall We Dance' 'Homegrown' 'The Commitments'
 'Love Is a Many-Splendored Thing' 'Wall Street']
Recommendation for user:  194
MovieIds:  ['The Paleface' 'Ong-Bak: The Thai Warrior'

Recommendation for user:  453
MovieIds:  ['Tomorrow Never Dies' 'The Wolverine' "The Sorcerer's Apprentice"
 'Justice League: Crisis on Two Earths' 'The Raven'
 'The Mortal Instruments: City of Bones'
 'The Hunger Games: Mockingjay - Part 2' 'Wizards' 'The Gate'
 'Conan the Destroyer']
Recommendation for user:  184
MovieIds:  ['Toy Story 3' 'Toy Story 2' 'Match Point' 'The Funhouse' 'Radio Days'
 'Rebel Without a Cause' 'Bound for Glory' 'Hollywood Ending'
 "What's Up, Tiger Lily?" 'The Messenger']
Recommendation for user:  357
MovieIds:  ['Toy Story 3' 'Rebel Without a Cause' 'Toy Story 2' 'Match Point'
 'Radio Days' 'The Funhouse' 'Bound for Glory' "White Men Can't Jump"
 'The Messenger' 'For Your Consideration']
Recommendation for user:  53
MovieIds:  ["Jacob's Ladder" 'Heat' 'Biloxi Blues' 'The Muppet Christmas Carol'
 'My Best Friend' 'Taxi to the Dark Side' 'Murmur of the Heart'
 'Combat Shock' 'Chinese Take-Out' 'Race to Witch Mountain']
Recommendation for user:  250
MovieIds:  

Recommendation for user:  609
MovieIds:  ['The Killing' 'Ghost Dog: The Way of the Samurai' 'The General'
 'Lucky Break' 'Bright Eyes' "Snake in the Eagle's Shadow"
 "The Cat's Meow" 'Driving Miss Daisy' 'Analyze This'
 "Jane Austen's Mafia!"]
Recommendation for user:  668
MovieIds:  ['O Lucky Man!' 'The Sting' 'Moll Flanders' 'Sex and Lucia'
 'Secondhand Lions' 'You Kill Me' 'Sliding Doors'
 'They Made Me a Criminal' 'Muppets Most Wanted' 'All or Nothing']
Recommendation for user:  210
MovieIds:  ['Faster, Pussycat! Kill! Kill!' 'Titus' 'Nowhere in Africa'
 'Dear Zachary: A Letter to a Son About His Father' 'The Dish'
 'The Artist' 'Dear Frankie' 'Naked' 'The Searchers'
 'Man Facing Southeast']
Recommendation for user:  158
MovieIds:  ['Little Miss Sunshine' 'Lock, Stock and Two Smoking Barrels'
 'The Manson Family' 'Thirteen' 'Dead Calm' "April Fool's Day"
 'Revenge of the Pink Panther' 'Exodus' 'Go' 'Time and Tide']
Recommendation for user:  127
MovieIds:  ['Mrs. Winterbourne' 'Roma

Recommendation for user:  115
MovieIds:  ['Love Me If You Dare' 'Casino Royale' 'Offside' 'Survive and Advance'
 'Wreck-It Ralph' 'Avalon' 'Rollerball' "Grandma's Boy"
 'Guardians of the Galaxy' 'Manhunter']
Recommendation for user:  272
MovieIds:  ['Malice' 'Match Point' 'Toy Story 3' 'The Messenger'
 "What's Up, Tiger Lily?" 'Hollywood Ending' 'Bound for Glory'
 'Africa Screams' 'Indecent Proposal' "White Men Can't Jump"]
Recommendation for user:  349
MovieIds:  ['South Park: Imaginationland' 'Johns' 'Black Knight' 'The Good Dinosaur'
 'Adventureland' 'Piranha 3DD' "National Lampoon's Vacation"
 'Land of the Lost' 'Meet the Deedles' 'Austenland']
Recommendation for user:  489
MovieIds:  ['Henry Fool' 'Young Sherlock Holmes' 'The Lawnmower Man'
 'The White Balloon' 'Faster, Pussycat! Kill! Kill!' 'Cronos'
 'Enchanted April' 'Wings of Desire' 'Ran' 'Prisoner of the Mountains']
Recommendation for user:  573
MovieIds:  ['The Green Mile' 'The Life of David Gale' 'Enter the Void' 'Chicago'

Recommendation for user:  377
MovieIds:  ['Wild Child' "The Devil's Backbone" 'The Longest Yard'
 'Prison Break: The Final Break' 'Miss Meadows' 'The Haunted Mansion'
 'Fame' 'A Month by the Lake' 'Two Mules for Sister Sara' 'Lolita']
Recommendation for user:  55
MovieIds:  ["Mr. Holland's Opus" 'Les Miserables' 'Braveheart'
 'Strawberry and Chocolate' 'Swimming with Sharks' 'Bitter Moon'
 'Forrest Gump' 'Red Rock West' 'Faster, Pussycat! Kill! Kill!'
 'Geronimo: An American Legend']
Recommendation for user:  329
MovieIds:  ['Les Miserables' 'The White Balloon' 'La Haine' 'Anne Frank Remembered'
 'Love & Human Remains' 'Burnt by the Sun' 'Faster, Pussycat! Kill! Kill!'
 'Cronos' 'Maya Lin: A Strong Clear Vision' 'I Shot Andy Warhol']
Recommendation for user:  459
MovieIds:  ['Toy Story 3' 'Toy Story 2' 'The Funhouse' 'Rebel Without a Cause'
 'Radio Days' 'Bound for Glory' 'Match Point' 'The Messenger'
 'Hollywood Ending' "White Men Can't Jump"]
Recommendation for user:  490
MovieIds:  

# Hybrid Approach C (Content-Based and User-Based Collaborative Filter)

In [71]:
df = pd.DataFrame()
df = pd.merge(user_cf_df, recom_content_pred, how= 'inner', left_on=['userId','movieId'], right_on = ['userId','movieId'])

In [72]:
df_test = pd.DataFrame()
df_test['predictions'] = df['predictions']
df_test['prediction'] = df['prediction']
x_test = df_test.values

In [73]:
#loading model pickle file
with open('hybrid_c.pickle', 'rb') as file:
    pickle_model = pickle.load(file)
    
Ypredict = pickle_model.predict(x_test)

In [74]:
df['hybrid_pred'] =  Ypredict 

In [75]:
df.head()

,userId,movieId,predictions,title_x,prediction,title_y,hybrid_pred
0,1,1,2.55,Toy Story,3.539915,Toy Story,3.425262
1,1,2,2.05,Jumanji,3.539915,Jumanji,3.369173
2,1,3,1.77,Grumpier Old Men,3.539915,Grumpier Old Men,3.337764
3,1,4,1.00,Waiting to Exhale,3.539915,Waiting to Exhale,3.251387
4,1,5,1.93,Father of the Bride Part II,3.539915,Father of the Bride Part II,3.355712


### Printing the recommendations for Hybrid Approach C

In [76]:
df= pd.merge(df, movie_title, how='left', left_on=['movieId'], right_on = ['movieId'])

In [77]:
sorted_pred = df.sort_values(by =['userId', 'hybrid_pred'], ascending= False)

In [78]:
users = df.userId.unique()
for i in range(len(users)):
    top10movies = sorted_pred.loc[(sorted_pred['userId'] == users[i])]['title'].values
    top10movies = top10movies[:10]
    print('Recommendation for user: ', users[i])
    print('MovieIds: ',top10movies)

Recommendation for user:  1
MovieIds:  ['Vampyros Lesbos' 'Dylan Moran: Monster' 'The King Is Alive' 'Innocence'
 'Maelström' 'Incendies' 'Cool as Ice'
 'The Looney, Looney, Looney Bugs Bunny Movie' 'Love Me If You Dare'
 'Seve']
Recommendation for user:  7
MovieIds:  ['Cronos' 'The Day the Sun Turned Cold' 'Prisoner of the Mountains'
 'Dream with the Fishes' 'Storefront Hitchcock' 'Taste of Cherry'
 'Vampyros Lesbos' 'Cool as Ice' 'The Art of War' 'Two Ninas']
Recommendation for user:  31
MovieIds:  ['My Super Ex-Girlfriend' 'Jason and the Argonauts' 'Creed' 'The Drop'
 'Born Rich' 'To Sir, with Love' 'Jack & Sarah' 'Elephant' 'Pollyanna'
 'The Bridges of Madison County']
Recommendation for user:  32
MovieIds:  ['The Paleface' 'Ong-Bak: The Thai Warrior' 'Pathfinder' 'Lamerica'
 'Mute Witness' 'Picture Bride' 'Cronos' 'Heavy'
 'The Day the Sun Turned Cold' 'Project S']
Recommendation for user:  36
MovieIds:  ['İtirazım Var' 'Star Trek V: The Final Frontier' 'Roadgames'
 'Mute Witness'

Recommendation for user:  19
MovieIds:  ['Mask' "The Ballad of Ramblin' Jack" 'Rudolph, the Red-Nosed Reindeer'
 'The Thin Red Line' 'Goodbye, Columbus' 'A Midnight Clear'
 'Picture Bride' 'Cronos' 'The Day the Sun Turned Cold' 'Project S']
Recommendation for user:  35
MovieIds:  ['Death of a Superhero' 'Vampyros Lesbos' 'Dylan Moran: Monster'
 'Excision' 'The King Is Alive' 'Innocence' 'Maelström' 'Incendies'
 'Cool as Ice' 'The Looney, Looney, Looney Bugs Bunny Movie']
Recommendation for user:  61
MovieIds:  ['Cronos' 'The Day the Sun Turned Cold' 'Prisoner of the Mountains'
 'Dream with the Fishes' 'Storefront Hitchcock' 'Taste of Cherry'
 'Vampyros Lesbos' 'Cool as Ice' 'The Art of War' 'Two Ninas']
Recommendation for user:  99
MovieIds:  ['A Midnight Clear' 'The Messenger' 'Indecent Proposal' 'The Funhouse'
 'Mask' 'Radio Days' 'Condorman' 'Bound for Glory' "White Men Can't Jump"
 'Hollywood Ending']
Recommendation for user:  102
MovieIds:  ['Cinderella' 'American Pop' 'Earthquake

Recommendation for user:  105
MovieIds:  ['Mulholland Falls' 'The Crush' 'Death Wish 2' 'Escape from L.A.'
 'The Cincinnati Kid' 'Biloxi Blues' 'Cronos'
 'The Day the Sun Turned Cold' 'Prisoner of the Mountains'
 'Dream with the Fishes']
Recommendation for user:  119
MovieIds:  ['A Midnight Clear' 'Goodbye, Columbus' 'The Thin Red Line'
 'Tai-Chi Master' 'Love! Valour! Compassion!' 'Mask'
 'For Your Consideration' 'Rudolph, the Red-Nosed Reindeer'
 'Picture Bride' 'Cronos']
Recommendation for user:  141
MovieIds:  ['Pusher' 'Cocaine Cowboys' 'End of Watch' 'Spy' 'Burn After Reading'
 'Toy Soldiers' 'Maria Full of Grace' 'Boys and Girls' 'Close Range'
 'Scarface']
Recommendation for user:  212
MovieIds:  ['Stepmom' 'Smiles of a Summer Night' 'Let It Ride' 'Maid in Manhattan'
 'Goodbye, Columbus' 'Prisoner of the Mountains' 'Dream with the Fishes'
 'Storefront Hitchcock' 'Vampyros Lesbos' 'Cool as Ice']
Recommendation for user:  233
MovieIds:  ['Arthur' 'Prisoner of the Mountains' 'Dream

Recommendation for user:  423
MovieIds:  ['Donnie Brasco' 'Things Change' 'İtirazım Var'
 'Ghost Dog: The Way of the Samurai' 'Bad Lieutenant' 'We Own the Night'
 'The Electric Horseman' 'Think Like a Man Too' 'Lamerica' 'Mute Witness']
Recommendation for user:  463
MovieIds:  ['Rudolph, the Red-Nosed Reindeer' 'Let It Ride' "Max Keeble's Big Move"
 'A Midnight Clear' 'The Thin Red Line' 'Mask' 'Goodbye, Columbus' 'Room'
 'The Day the Sun Turned Cold' 'Prisoner of the Mountains']
Recommendation for user:  522
MovieIds:  ['Prisoner of the Mountains' 'Dream with the Fishes'
 'Storefront Hitchcock' 'Vampyros Lesbos' 'Cool as Ice' 'The Art of War'
 'Two Ninas' 'The King Is Alive' 'Erik the Viking' 'Innocence']
Recommendation for user:  580
MovieIds:  ['Smiles of a Summer Night' 'The Funhouse' 'Room' 'The Thin Red Line'
 'Malice' 'Rebel Without a Cause' "What's Up, Tiger Lily?" 'Match Point'
 'The Messenger' 'Condorman']
Recommendation for user:  585
MovieIds:  ['Room' 'Stepmom' 'Smiles of 

Recommendation for user:  152
MovieIds:  ["Max Keeble's Big Move" 'Stepmom' 'A Midnight Clear' 'Tai-Chi Master'
 'Radio Days' 'Cronos' 'The Day the Sun Turned Cold'
 'Prisoner of the Mountains' 'Dream with the Fishes'
 'Storefront Hitchcock']
Recommendation for user:  257
MovieIds:  ['Tea with Mussolini' 'Mute Witness' 'Picture Bride' 'Cronos'
 'The Day the Sun Turned Cold' 'Project S' 'Female Perversions'
 'Angel Baby' 'Prisoner of the Mountains' 'Losing Chase']
Recommendation for user:  265
MovieIds:  ['42nd Street' 'Muppet Treasure Island' 'S1m0ne' 'The Artist'
 'Straight-Jacket' 'Trouble in Paradise' 'Men with Guns'
 'What Just Happened' 'Dancing in September' 'Find Me Guilty']
Recommendation for user:  309
MovieIds:  ['Hotel Chevalier' 'Like Water for Chocolate' 'Four Lions' 'Daylight'
 'The Dangerous Lives of Altar Boys' 'The Lords of Flatbush'
 'Throne of Blood' 'Odds Against Tomorrow' 'Nightcrawler' 'High and Low']
Recommendation for user:  344
MovieIds:  ['The Thin Red Line' '

...

Recommendation for user:  549
MovieIds:  ['Prisoner of the Mountains' 'Dream with the Fishes'
 'Storefront Hitchcock' 'Vampyros Lesbos' 'Cool as Ice' 'The Art of War'
 'Two Ninas' 'The King Is Alive' 'Erik the Viking' 'Innocence']
Recommendation for user:  556
MovieIds:  ['Michael Clayton' 'Fire with Fire' 'Lamerica' 'The White Balloon'
 'Last Summer in the Hamptons' 'La Haine' 'Anne Frank Remembered'
 'Chungking Express' 'Angela' 'The Amazing Panda Adventure']
Recommendation for user:  567
MovieIds:  ['Lamerica' 'The White Balloon' 'Last Summer in the Hamptons' 'La Haine'
 'Anne Frank Remembered' 'The Amazing Panda Adventure'
 'Love & Human Remains' 'Mute Witness' 'Nadja' 'Exotica']
Recommendation for user:  572
MovieIds:  ['Lantana' 'Hotel Chevalier' 'Odds Against Tomorrow' 'Nightcrawler'
 'Throne of Blood' 'White Heat' 'The Sting' 'Four Lions'
 'The Dangerous Lives of Altar Boys' 'Daylight']
Recommendation for user:  620
MovieIds:  ['Bob le Flambeur' 'Odds Against Tomorrow' 'Hotel C

Recommendation for user:  58
MovieIds:  ['Too Big to Fail' 'The Great Santini' 'Mildred Pierce' 'Padre Padrone'
 'Sleuth' 'Georgia' 'Shaft' "Don't Tell Mom the Babysitter's Dead"
 'Maggie' 'Kansas']
Recommendation for user:  63
MovieIds:  ['The Funhouse' 'Toy Story 3' 'Rebel Without a Cause' 'Tai-Chi Master'
 'Radio Days' 'The Messenger' 'Condorman' 'Bound for Glory'
 "White Men Can't Jump" 'Hollywood Ending']
Recommendation for user:  142
MovieIds:  ['Toy Story 3' 'Rebel Without a Cause' 'Radio Days' 'The Messenger'
 'Rudolph, the Red-Nosed Reindeer' 'Condorman' "White Men Can't Jump"
 'For Your Consideration' 'Match Point' 'Africa Screams']
Recommendation for user:  146
MovieIds:  ['The Funhouse' 'Toy Story 3' 'Toy Story 2' 'Radio Days' 'The Messenger'
 'Bound for Glory' "White Men Can't Jump" 'Hollywood Ending' 'Match Point'
 'For Your Consideration']
Recommendation for user:  273
MovieIds:  ['Mask' 'The Funhouse' 'Tai-Chi Master' 'Radio Days' 'The Messenger'
 'Rudolph, the Red-Nose

Recommendation for user:  112
MovieIds:  ['The Funhouse' 'Toy Story 3' 'Toy Story 2' 'Rebel Without a Cause'
 'Radio Days' 'The Messenger' 'Condorman' 'Bound for Glory'
 "White Men Can't Jump" 'Hollywood Ending']
Recommendation for user:  181
MovieIds:  ['O Lucky Man!' 'You Kill Me' 'The Sting' 'To Live and Die in L.A.'
 'Sliding Doors' 'Muppets Most Wanted' 'The Enforcer' 'Sex and Lucia'
 'Secondhand Lions' '11:14']
Recommendation for user:  208
MovieIds:  ['Aspen' 'Gladiator 1992' 'Colonel Chabert' 'Straight to Hell' 'Gothic'
 'The Kid Brother' 'The Women' 'Big Trouble' 'Party Girl'
 'After the Sunset']
Recommendation for user:  274
MovieIds:  ['Picture Bride' 'Cronos' 'The Day the Sun Turned Cold' 'Angel Baby'
 'Prisoner of the Mountains' 'Dream with the Fishes' 'Gabbeh'
 'Storefront Hitchcock' 'Taste of Cherry' 'Vampyros Lesbos']
Recommendation for user:  278
MovieIds:  ['The Chaser' 'Gold Diggers: The Secret of Bear Mountain'
 'Paris is Burning' 'Dark Passage' 'Little Man Tate' 'M

Recommendation for user:  469
MovieIds:  ['The Tall Blond Man with One Black Shoe' 'Electric Dreams' 'Moon'
 'The Girl with the Dragon Tattoo' 'The Matrix' 'The Social Network'
 '2001: A Space Odyssey' '28 Days Later' 'Pirates of Silicon Valley'
 'Contagion']
Recommendation for user:  115
MovieIds:  ['Love Me If You Dare' 'Survive and Advance' 'Rollerball' 'Offside'
 "Grandma's Boy" 'The Last Starfighter' 'Casino Royale' 'Avalon'
 'Guardians of the Galaxy' 'Peter Pan']
Recommendation for user:  272
MovieIds:  ['The Funhouse' 'Toy Story 3' 'Rebel Without a Cause' 'Toy Story 2'
 'Radio Days' 'The Messenger' "Harry Potter and the Philosopher's Stone"
 'Condorman' 'Bound for Glory' "White Men Can't Jump"]
Recommendation for user:  349
MovieIds:  ['Johns' 'South Park: Imaginationland' 'The Good Dinosaur'
 'Jurassic World' "National Lampoon's Vacation" 'Adventureland'
 'The Land Before Time' 'T-Rex: Back to the Cretaceous' 'Caddyshack II'
 'Austenland']
Recommendation for user:  489
MovieIds

Recommendation for user:  65
MovieIds:  ['The Thief of Bagdad' 'Top Secret!' '20,000 Leagues Under the Sea'
 'Stargate: Continuum' 'The 7th Voyage of Sinbad' 'The Princess Bride'
 'Sinbad and the Eye of the Tiger' 'Iron Man'
 'History of the World: Part I' 'The Princess Diaries 2: Royal Engagement']
Recommendation for user:  638
MovieIds:  ['Enter the Dragon' 'The Day the Sun Turned Cold' 'Taste of Cherry'
 'Vampyros Lesbos' 'Cool as Ice' 'The Art of War' 'The King Is Alive'
 'Innocence' 'Maelström' 'The Looney, Looney, Looney Bugs Bunny Movie']
Recommendation for user:  14
MovieIds:  ['Vampyros Lesbos' 'Cool as Ice' 'The King Is Alive' 'Innocence'
 'Maelström' 'The Looney, Looney, Looney Bugs Bunny Movie' 'Incendies'
 'Dylan Moran: Monster' 'Seve' 'Land of Silence and Darkness']
Recommendation for user:  269
MovieIds:  ['Picture Bride' 'The Day the Sun Turned Cold' 'Angel Baby' 'Gabbeh'
 'Taste of Cherry' 'Man Facing Southeast' 'Vampyros Lesbos' 'Cool as Ice'
 'The Art of War' 'The Ki